<a href="https://colab.research.google.com/github/Alesapo-py/File_CDXML_mol_13667/blob/main/RDKit_OMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files # Importo i file utili su colab, con formato zip
uploaded = files.upload()

Saving arial-font.zip to arial-font.zip
Saving pycdxml-master.zip to pycdxml-master.zip


In [4]:
%%capture
!unzip /content/arial-font.zip

In [3]:
%%capture
!unzip /content/CDX.zip

In [4]:
%%capture
!unzip /content/p130.zip

In [5]:
%%capture
!unzip /content/p131.zip

In [5]:
%%capture
!unzip /content/pycdxml-master.zip # convertitore cdx cdxml

In [7]:
!pip install rdkit --upgrade # installo RDKit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 46.7 MB/s eta 0:00:00


In [6]:
%%capture
!pip install -e /content/pycdxml-master/ #installo il convertitore

In [7]:
import sys
sys.path.append('/content/pycdxml-master/pycdxml')
sys.path.append('/content/pycdxml-master')

In [8]:
from pycdxml import cdxml_converter

In [9]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdAbbreviations
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole

In [10]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from lxml import etree as ET
import matplotlib.font_manager
import unicodedata

In [11]:
def convert_cdx_to_cdxml(input_file_path, output_file_path):
  cdxml_converter.write_cdxml_file(cdxml_converter.read_cdx(input_file_path), output_file_path)

In [12]:
def color_string(string, color):
  #Colora una stringa usando i codici ANSI escape.
  color_codes = {
      "black": 30,
      "red": 31,
      "green": 32,
      "yellow": 33,
      "blue": 34,
      "magenta": 35,
      "cyan": 36,
      "white": 37,
      "reset": 0,  # Reimposta il colore al default
  }
  color_code = color_codes.get(color.lower(), color_codes["reset"])
  return f"\033[{color_code}m{string}\033[0m"  # Applica il codice colore

In [13]:
def move_fragment_into_group(cdxml_file, output_file):
    # Carica il file CDXML
    tree = ET.parse(cdxml_file)
    root = tree.getroot()
    # Trova tutti i tag <group> e <fragment>
    groups = root.findall(".//group")
    fragments = root.findall(".//fragment")
    # Funzione per trovare il genitore di un tag
    def find_parent(root, child):
        for parent in root.iter():
            if child in parent:
                return parent
        return None
    # Funzione per controllare se un frammento è discendente di un tag <group>
    def is_descendant_of_group(fragment):
        parent = fragment
        while parent is not None:
            parent = find_parent(root, parent)
            if parent is not None and parent.tag == "group":
                return True
        return False
    # Per ogni tag group, vediamo se c'è un fragment da associare
    for group in groups:
        group_id = group.get("id")
        # Cerca i frammenti che NON sono già figli di un tag <group>
        for fragment in fragments:
            frag_id = fragment.get("id")
            # Controlla se il fragment è già parte di un tag <group>
            # o se è annidato in un altro fragment
            if not is_descendant_of_group(fragment):
                # Trova il genitore effettivo del fragment
                parent = find_parent(root, fragment)
                if parent is not None:
                    print(color_string(f"Moving fragment {frag_id} into group {group_id}",color='red'))
                    parent.remove(fragment)  # Rimuovi il fragment dalla sua posizione attuale
                    group.append(fragment)   # Aggiungi il fragment all'interno del tag group
                else:
                    print(color_string(f"Parent not found for fragment {frag_id}",color='blue'))
            else:
                print(color_string(f"Fragment {frag_id} is already inside a group. No need to move.",color='green'))
    # Salva il file CDXML modificato
    tree.write(output_file, encoding="utf-8", xml_declaration=True)
    print(f"Modifications saved to {output_file}")

In [14]:
def abbrevs_molecule(molobj):
    # abbreviazioni personalizzate
    defn = '''
    CN      C#N    CN    NC
    CO<sub>2</sub>H    C(=O)O    CO<sub>2</sub>H    HO<sub>2</sub>C
    SO<sub>3</sub>H    S(=O)(=O)[OH]    SO<sub>3</sub>H    HO<sub>3</sub>S
    N<sub>3</sub>      [N-]=[N+]=N    N<sub>3</sub>    N<sub>3</sub>
    CHO     C=O    CHO    OHC
    NC      N#C    NC    CN
    OCH<sub>3</sub>    OC    OCH<sub>3</sub>    H<sub>3</sub>CO
    NO<sub>2</sub>     [N+](=O)[O-]    NO<sub>2</sub>    O<sub>2</sub>N
    SO<sub>3</sub>     S(=O)(=O)[O-]    SO<sub>3</sub><sup>-</sup>    <sup>-</sup>O<sub>3</sub>S
    CO<sub>2</sub>     C(=O)O    CO<sub>2</sub>    O<sub>2</sub>C
    PO<sub>3</sub>H<sub>2</sub>   OP(=O)(O)O    PO<sub>3</sub>H<sub>2</sub>    H<sub>2</sub>O<sub>3</sub>P
    CF<sub>3</sub>    C(F)(F)F    CF<sub>3</sub>    F<sub>3</sub>C
    '''
    # inizializzazione delle abbreviazioni personalizzate
    abbrevs = rdAbbreviations.ParseAbbreviations(defn)
    # applicazione delle abbreviazioni personalizzate
    lm = rdAbbreviations.CondenseMolAbbreviations(molobj, abbrevs, maxCoverage=1.0)
    return lm # molecola con le abbreviazioni applicate

In [15]:
# Funzione essenziale per trovare l'indice atomico di rdkit, dando in input
# la tupla con le coordinate corrispondenti.
def trova_chiave(dizionario, tupla_da_cercare):
    for key, value in dizionario.items():
        if value == tupla_da_cercare:
            return key
    return None

In [16]:
# converte le values del dizionario contenenti le coordinate del cdxml
# da stringhe in delle tuple con due valori float.
def convert_string_to_float_tuple(coord_dict_with_strings):
  coord_dict_with_floats = {}
  for key, value in coord_dict_with_strings.items():
    x_str, y_str = value.split()
    x = float(x_str)
    y = float(y_str)
    coord_dict_with_floats[key] = (x, y)
  return coord_dict_with_floats

In [17]:
# serve a trovare i fattori di scala per convertire le coordinate degli atomi
# date da rdkit nei valori originari del file cdxml.
# così da convertire il dizionario con gli atomi di rdkit, rendendo univoche le values,
# che contengono le coordinate, per ambedue i dizionari.
def trova_fattore_di_scala(diz1, diz2):
    # Ottieni la prima key e la sua tupla associata nel primo dizionario
    prima_key_diz1 = next(iter(diz1))
    coord_diz1 = diz1[prima_key_diz1]
    # Ottieni la prima key e la sua tupla associata nel secondo dizionario
    prima_key_diz2 = next(iter(diz2))
    coord_diz2 = diz2[prima_key_diz2]
    # Estrai le coordinate x e y
    x1, y1 = coord_diz1
    x2, y2 = coord_diz2
    # Calcola i rapporti tra le coordinate x e y, che saranno i fattori di scala
    rapporto_x = x1 / x2
    rapporto_y = y1 / y2
    return rapporto_x, rapporto_y

In [18]:
def Retain_Stereo_and_map_fragments(molobj,pathcdxml,verboso):
  conf = molobj.GetConformer(0) # genera un conformero dal molobj
  #dizionario con indici di rdkit e coordinate rdkit
  coord_dict= {} # inizializzazione dizionario
  for atom in molobj.GetAtoms(): # itera su ogni atomo del conformero
    pos = conf.GetAtomPosition(atom.GetIdx()) #estrae le coordinate
    x, y, _ = pos  #ignora la coordinata Z con '_'
    # per ogni atomo salva nel dizionario l'indice atomico come chiave
    # e le coordinate come valori
    coord_dict[atom.GetIdx()] = (x, y)
  tree = ET.parse(pathcdxml) # inizializza il CDXML per poterlo leggere
  root = tree.getroot()
  atoms = root.findall('.//n') # trova i nodi all'interno del file CDXML
  #dizionario con indici atomici CDXML e loro coordinate
  atomcoords_info_cdxml = {} # inizializzazione dizionario
  for atom in atoms: # itera su tutti i nodi
    atom_id = int(atom.get('id'))  # Ottieni l'id dell'atomo
    coords = atom.get('p')  # Ottieni le coordinate
    atomcoords_info_cdxml[atom_id] = coords # salva gli elementi nel dizionario
  #converto le coordinate del dizionario cdxml in una tupla con numeri float
  atomcoords_info_cdxml_tupla=convert_string_to_float_tuple(atomcoords_info_cdxml)
  #converto le coordinate di rdkit in cdxml con il fattore di scala
  rapporto_x, rapporto_y=trova_fattore_di_scala(atomcoords_info_cdxml_tupla,coord_dict)
  coord_dict_scaled = {k: (round(v[0] * rapporto_x, 2), round(v[1] * rapporto_y, 2)) for k, v in coord_dict.items()}
  #definisco un dizionario che ha come keys gli indici di rdkit, come values quelli cdxml.
  dict_comparative_idx={} # inizializzazione dizionario
  # iterazione sugli elementi del dizionario RDKit scalato e CDXML
  for key_coord_dict, value_coord_dict in coord_dict_scaled.items():
    for key_atomcoords_dict, value_atomcoords_dict in atomcoords_info_cdxml_tupla.items():
      # se le coordinate tra questi coincidono salvo gli indici nel dizionario comparativo
      if value_coord_dict == value_atomcoords_dict:
        dict_comparative_idx[key_coord_dict] = key_atomcoords_dict
        break
  # definisco un dizionario che abbia come keys gli id degli atomi frammento, e come values,
  # gli id degli atomi appartenenti al frammento
  fragments_dict = {} # inizializzazione dizionario
  # Iterazione su tutti i nodi <n> che rappresentano frammenti (NodeType='Fragment')
  for fragment_node in root.findall('.//n[@NodeType="Fragment"]'):
      fragment_id = fragment_node.get('id')
      # Inizializzo una lista per memorizzare gli atomi presenti all'interno di questo frammento
      atoms_in_fragment = []
      # Trova il nodo <fragment> all'interno del frammento
      fragment_content = fragment_node.find('./fragment')
      if fragment_content is not None:
          # Cicla su tutti gli atomi (nodi <n>) all'interno del nodo <fragment>
          for atom in fragment_content.findall('.//n'):
              # Estrai l'id dell'atomo e altre proprietà utili
              atom_id = atom.get('id')
              # Aggiungi l'atomo alla lista degli atomi per questo frammento
              atoms_in_fragment.append(int(atom_id))
      # Aggiungi al dizionario il frammento con la lista degli atomi annidati
      fragments_dict[int(fragment_id)] = atoms_in_fragment
  dict_4={} #dizionario con le coordinate dei frammenti e rispettivi indici
  for atom in root.findall('.//n'):
      node_type = atom.get('NodeType')
      if node_type=='Fragment':
          coords = atom.get('p')
          atom_id = atom.get('id')
          dict_4[int(atom_id)] = (coords)
  dict_4=convert_string_to_float_tuple(dict_4)
  global_atoms=[] #atomi globali fuori dai frammenti
  for bond in root.findall('.//b'): #itero su tutti i legami della molecola sul file cdxml
    atomo_iniziale = int(bond.get('B')) #salvo l'atomo Begin coinvolto nel legame
    atomo_finale = int(bond.get('E')) #salvo l'atomo end coinvolto nel legame
    bond_id = bond.get('id')
    if atomo_iniziale and atomo_finale in dict_4.keys():
      # se l'atomo begin non è l'atomo del frammento, sarà quello globale
      # allora aggiungo alla lista  l'atomo Begin
      if atomo_iniziale not in dict_4.keys():
        global_atoms.append(atomo_iniziale)
      # se l'atomo End non è l'atomo del frammento, sarà quello globale
      # allora aggiungo alla lista  l'atomo End
      if atomo_finale not in dict_4.keys():
        global_atoms.append(atomo_finale)
  # dizionario con gli indici degli atomi globali, e rispettive coordinate
  dict_5={} # inizializzazione dizionario
  for atom in root.findall('.//n'): # iterazione atomi nodo
    atom_id = int(atom.get('id')) # ricavo id atomico CDXML
    coords = atom.get('p') # ricavo coordinate atomiche CDXML
    if atom_id in global_atoms: # se è un atomo globale, lo salvo
      dict_5[atom_id]=coords
  dict_5=convert_string_to_float_tuple(dict_5)
  #genero una lista con tutti gli indici degli atomi annidati nei frammenti
  list_idx=[]
  for idx in fragments_dict.values():
      list_idx.extend(idx)
  # dizionario con gli indici degli atomi relativi ai frammenti
  # e rispettive coordinate senza gli atomi globali relativi
  dict_6={}
  for atom in root.findall('.//n'): # iterazione atomi nodo
    atom_id = int(atom.get('id')) # ricavo id atomico CDXML
    coords = atom.get('p') # ricavo coordinate atomiche CDXML
    NodeType = atom.get('NodeType') # ricavo il NodeType
    if atom_id in list_idx:
      if NodeType==None:
        dict_6[atom_id]=coords
  dict_6=convert_string_to_float_tuple(dict_6)
  # genero un dizionario con gli atomi globali annidati nei frammenti e rispettive coordinate
  dict_7={}
  for atom in root.findall('.//n'): # iterazione atomi nodo
    atom_id = atom.get('id') # ricavo id atomico CDXML
    coords = atom.get('p') # ricavo coordinate atomiche CDXML
    NodeType = atom.get('NodeType') # ricavo il NodeType
    if NodeType=='ExternalConnectionPoint':
      dict_7[int(atom_id)]=coords
  dict_7=convert_string_to_float_tuple(dict_7)
  #dizionario che ha come keys l'id dei frammenti e come value gli id degli atomi globali annidati
  dict_8 = {}
  for atom in root.findall('.//n'): # iterazione atomi nodo
    atom_id = atom.get('id') # ricavo id atomico CDXML
    NodeType = atom.get('NodeType') # ricavo il NodeType
    if NodeType == 'Fragment': # se il NodeType è fragment
      atomo_frammento = atom
      frammento_id = int(atom.get('id'))
      for child in atom.findall('.//n'):  # Cerca tutti gli atomi all'interno del frammento
        atomo_annidato_id = int(child.get('id'))
        NodeType = child.get('NodeType')
        if NodeType == 'ExternalConnectionPoint':
          dict_8[frammento_id] = atomo_annidato_id
  # dizionario con i frammenti id come keys e gli atomi globali id come values
  dict_9={}
  for bond in root.findall('.//b'):
    atomo_iniziale = int(bond.get('B'))
    atomo_finale = int(bond.get('E'))
    bond_id = bond.get('id')
    if atomo_iniziale in dict_8.keys(): #se l'atomo end non è l'atomo del frammento, sarà quello globale
      dict_9[atomo_iniziale]=atomo_finale
    if atomo_finale in dict_8.keys(): #se l'atomo begin non è l'atomo del frammento, sarà quello globale
      dict_9[atomo_finale]=atomo_iniziale
  #matching dei dizionari 8 e 9 per generare un nuovo dizionario che avrà
  #come keys gli id degli atomi globali annidati e come values gli id degli atomi globali
  dict_10={}
  for key8 in dict_8.keys():
    value_8=dict_8[key8]
    for key9 in dict_9.keys():
      if key8==key9:
        value_9=dict_9[key9]
        dict_10[value_8]=value_9
  def print_dict_vertical(dictionary, title):
        print(color_string(f"{title}:", color='blue'))
        for key, value in dictionary.items():
            print(f"  {key}: {value}")
  if verboso:
    print_dict_vertical(coord_dict_scaled, 'coord_dict_scaled')
    print_dict_vertical(atomcoords_info_cdxml_tupla, 'atomcoords_info_cdxml_tupla')
    print_dict_vertical(dict_comparative_idx, 'dict_comparative')
    print_dict_vertical(fragments_dict, 'fragments_dict')
    print_dict_vertical(dict_4, 'dict_4')
    print_dict_vertical(dict_10, 'dict_10')
    print_dict_vertical(dict_7, 'dict_7')
#dopo aver definito i dizionari, la funzione a tal punto, può modificare i legami
#secondo la corretta stereochimica, modificando anche le coordinate degli atomi annidati
#nel dizionario di rdkit e nel molobj di rdkit
  tree = ET.parse(pathcdxml)
  root = tree.getroot()
  for bond in root.findall('.//b'):
    atomo_iniziale = int(bond.get('B'))
    atomo_finale = int(bond.get('E'))
    display = bond.get('Display')
    if display == 'Dash':
      print(color_string(f'legame tra {atomo_iniziale} e {atomo_finale}, Dash','red'))
      coords_x_y_Iniziale= atomcoords_info_cdxml_tupla[atomo_iniziale]
      coords_x_y_Finale= atomcoords_info_cdxml_tupla[atomo_finale]
      print(f'le coordinate dell\'atomo finale sono: {coords_x_y_Finale}')
      print(f'le coordinate dell\'atomo iniziale sono: {coords_x_y_Iniziale}')
    #controllo se l'atomo coinvolto nel legame dash è un atomo globale annidato, con il parametro
    #Nodetype='ExternalconnectionPoint
      if atomo_iniziale in dict_10.keys():
        atomo_iniziale = dict_10[atomo_iniziale]
        coords_x_y_Iniziale= atomcoords_info_cdxml_tupla[atomo_iniziale]
        print(f'le coordinate dell\'atomo iniziale sono: {coords_x_y_Iniziale}')
        print(color_string(f'sostituisco l\'atomo inziale, con quello globale nel legame Dash: {atomo_iniziale}',color='green'))
      if atomo_finale in dict_10.keys():
        atomo_finale = dict_10[atomo_finale]
        coords_x_y_Finale= atomcoords_info_cdxml_tupla[atomo_finale]
        print(f'le coordinate dell\'atomo finale sono: {coords_x_y_Finale}')
        print(color_string(f'sostituisco l\'atomo finale, con quello globale nel legame Dash: {atomo_finale}',color='green'))
    #controllo se l'atomo coinvolto nel legame è un atomo frammento, se lo fosse
    #salvo le sue coordinate, per incollarle nel dizionario coord_dict_scaled di rdkit
      if atomo_iniziale in fragments_dict.keys():
        #salvo le coordinate dell'atomo frammento, che cdxml coinvolge nel legame
        coords_x_y_Iniziale= dict_4[atomo_iniziale]
        print(f'le coordinate da sostituire sono: {coords_x_y_Iniziale}')
        #cerco il primo atomo del frammento, che è coinvolto nel legame secondo rdkit
        atomo_iniziale=fragments_dict[atomo_iniziale] [0]
        print(f'atomo iniziale corrispondente al frammento id nel legame dash: {atomo_iniziale}')
        #cerco l'indice dell'atomo rdkit a cui incollare le coordinate fammento
        atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_iniziale)
        if atom_idx_rdkit_da_cambiare is None:
          print('non trovato')
        else:
          print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
          #aggiorno le coordinate al dict di rdkit
          coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Iniziale
          print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
          conf = molobj.GetConformer()# incollo le nuove coordinate nell'atomo di rdkit
          new_x, new_y = coords_x_y_Iniziale
          new_x=new_x/rapporto_x
          new_y=new_y/rapporto_y
          new_z = 0.0
          conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
          updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
          print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
      if atomo_finale in fragments_dict.keys():
        #salvo le coordinate dell'atomo frammento, che cdxml coinvolge nel legame
        coords_x_y_Finale= dict_4[atomo_finale]
        print(f'le coordinate da sostituire sono: {coords_x_y_Finale}')
        #cerco il primo atomo del frammento, che è coinvolto nel legame secondo rdkit
        atomo_finale=fragments_dict[atomo_finale] [0]
        print(f'atomo finale corrispondente al frammento id nel legame dash: {atomo_finale}')
        #cerco l'indice dell'atomo rdkit a cui incollare le coordinate fammento
        atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_finale)
        if atom_idx_rdkit_da_cambiare is None:
          print('non trovato')
        else:
          print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
          #aggiorno le coordinate al dict di rdkit
          coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Finale
          print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
          conf = molobj.GetConformer()# incollo le nuove coordinate nell'atomo di rdkit
          new_x, new_y = coords_x_y_Finale
          new_x=new_x/rapporto_x
          new_y=new_y/rapporto_y
          new_z = 0.0
          conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
          updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
          print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
    #controllo se l'atomo coinvolto nel legame sia il primo atomo annidato di un frammento, quindi
    #devo aggiornarlo con le coordinate del frammento.
      for lista_atomi in fragments_dict.values():
        if atomo_iniziale == lista_atomi[0]:# se l'atomo è il primo della lista frammenti
          print(f'atomo iniziale da cui trovare il referee: {atomo_iniziale}')
          #prende l'atomo frammento come referenza
          atomo_referee= trova_chiave(fragments_dict, lista_atomi)
          print(f'atomo referee: {atomo_referee}')
          #prende le coordinate dell'atomo frammento referente
          coords_x_y_Iniziale= atomcoords_info_cdxml_tupla[atomo_referee]
          print(f'le coordinate dell\'atomo iniziale sono: {coords_x_y_Iniziale}')
          #cerco l'indice dell'atomo rdkit a cui incollare le coordinate fammento
          atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_iniziale)
          if atom_idx_rdkit_da_cambiare is None:
            print('non trovato')
          else:
            print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
            #aggiorno le coordinate al dict di rdkit
            coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Iniziale
            print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
            conf = molobj.GetConformer()#incollo le nuove coordinate nell'atomo di rdkit
            new_x, new_y = coords_x_y_Iniziale
            new_x=new_x/rapporto_x
            new_y=new_y/rapporto_y
            new_z = 0.0
            conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
            updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
            print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
        else:
          None
      for lista_atomi in fragments_dict.values():
        if atomo_finale == lista_atomi[0]:# se l'atomo è il primo della lista frammenti
          print(f'atomo finale da cui trovare il referee: {atomo_finale}')
          #prende l'atomo frammento come referenza
          atomo_referee= trova_chiave(fragments_dict, lista_atomi)
          print(f'atomo referee: {atomo_referee}')
          #prende le coordinate dell'atomo frammento referente
          coords_x_y_Finale= atomcoords_info_cdxml_tupla[atomo_referee]
          print(f'le coordinate dell\'atomo finale sono: {coords_x_y_Finale}')
          #cerco l'indice dell'atomo rdkit a cui incollare le coordinate fammento
          atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_finale)
          if atom_idx_rdkit_da_cambiare is None:
            print('non trovato')
          else:
            print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
            #aggiorno le coordinate al dict di rdkit
            coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Finale
            print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
            conf = molobj.GetConformer()#incollo le nuove coordinate nell'atomo di rdkit
            new_x, new_y = coords_x_y_Finale
            new_x=new_x/rapporto_x
            new_y=new_y/rapporto_y
            new_z = 0.0
            conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
            updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
            print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
        else:
          None
#Modifica della stereochimica se il legame è dash
      atomo_iniziale_rdkit=trova_chiave(coord_dict_scaled, coords_x_y_Iniziale)
      print(f'atomo iniziale: {atomo_iniziale_rdkit}')
      atomo_finale_rdkit=trova_chiave(coord_dict_scaled, coords_x_y_Finale)
      print(f'atomo finale: {atomo_finale_rdkit}')
      if atomo_iniziale_rdkit is None or atomo_finale_rdkit is None:
        print(color_string(f'errore: atomo iniziale cdxml {atomo_iniziale}={atomo_iniziale_rdkit}',color='red'))
        print(color_string(f'errore: atomo finale cdxml {atomo_finale}={atomo_finale_rdkit}',color='red'))
      else:
        legami_stereo = molobj.GetBondBetweenAtoms(atomo_iniziale_rdkit,atomo_finale_rdkit)
        if legami_stereo is None:
          print(color_string(f'non ci sono legami stereo tra {atomo_iniziale_rdkit} e {atomo_finale_rdkit}',color='red'))
        else:
          print(color_string(f'ci sono legami stereo Dash tra {atomo_iniziale_rdkit} e {atomo_finale_rdkit} imposto STEREOANY e BEGINDASH',color='blue'))
          legami_stereo.SetStereo(Chem.rdchem.BondStereo.STEREOANY)
          legami_stereo.SetBondDir(Chem.rdchem.BondDir.BEGINDASH)
#se il legame è wedge, fa le stesse cose del codice sovrastante, ma imposta il legame wedge
    elif display in ['WedgeEnd', 'WedgeBegin']:
      print(color_string(f'legame tra {atomo_iniziale} e {atomo_finale}, Wedge','yellow'))
      coords_x_y_Iniziale= atomcoords_info_cdxml_tupla[atomo_iniziale]
      coords_x_y_Finale= atomcoords_info_cdxml_tupla[atomo_finale]
      print(f'le coordinate dell\'atomo finale sono: {coords_x_y_Finale}')
      print(f'le coordinate dell\'atomo iniziale sono: {coords_x_y_Iniziale}')
      if atomo_iniziale in dict_10.keys():
        atomo_iniziale = dict_10[atomo_iniziale]
        coords_x_y_Iniziale= atomcoords_info_cdxml_tupla[atomo_iniziale]
        print(f'le coordinate dell\'atomo iniziale sono: {coords_x_y_Iniziale}')
        print(color_string(f'sostituisco l\'atomo inziale, con quello globale nel legame Wedge: {atomo_iniziale}',color='green'))
      if atomo_finale in dict_10.keys():
        atomo_finale = dict_10[atomo_finale]
        coords_x_y_Finale= atomcoords_info_cdxml_tupla[atomo_finale]
        print(f'le coordinate dell\'atomo finale sono: {coords_x_y_Finale}')
        print(color_string(f'sostituisco l\'atomo finale, con quello globale nel legame Wedge: {atomo_finale}',color='green'))
      if atomo_iniziale in fragments_dict.keys():
        coords_x_y_Iniziale= dict_4[atomo_iniziale]
        print(f'le coordinate da sostituire sono: {coords_x_y_Iniziale}')
        atomo_iniziale=fragments_dict[atomo_iniziale] [0]
        print(f'atomo iniziale corrispondente al frammento id nel legame Wedge: {atomo_iniziale}')
        atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_iniziale)
        if atom_idx_rdkit_da_cambiare is None:
          print('non trovato')
        else:
          print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
          coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Iniziale
          print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
          conf = molobj.GetConformer()
          new_x, new_y = coords_x_y_Iniziale
          new_x=new_x/rapporto_x
          new_y=new_y/rapporto_y
          new_z = 0.0
          conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
          updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
          print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
      if atomo_finale in fragments_dict.keys():
        coords_x_y_Finale= dict_4[atomo_finale]
        print(f'le coordinate da sostituire sono: {coords_x_y_Finale}')
        atomo_finale=fragments_dict[atomo_finale] [0]
        print(f'atomo finale corrispondente al frammento id nel legame Wedge: {atomo_finale}')
        atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_finale)
        if atom_idx_rdkit_da_cambiare is None:
          print('non trovato')
        else:
          print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
          coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Finale
          print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
          conf = molobj.GetConformer()
          new_x, new_y = coords_x_y_Finale
          new_x=new_x/rapporto_x
          new_y=new_y/rapporto_y
          new_z = 0.0
          conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
          updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
          print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
      for lista_atomi in fragments_dict.values():
        if atomo_iniziale == lista_atomi[0]:
          print(f'atomo iniziale da cui trovare il referee: {atomo_iniziale}')
          atomo_referee= trova_chiave(fragments_dict, lista_atomi)
          print(f'atomo referee: {atomo_referee}')
          coords_x_y_Iniziale= atomcoords_info_cdxml_tupla[atomo_referee]
          print(f'le coordinate dell\'atomo iniziale sono: {coords_x_y_Iniziale}')
          atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_iniziale)
          if atom_idx_rdkit_da_cambiare is None:
            print('non trovato')
          else:
            print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
            coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Iniziale
            print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
            conf = molobj.GetConformer()
            new_x, new_y = coords_x_y_Iniziale
            new_x=new_x/rapporto_x
            new_y=new_y/rapporto_y
            new_z = 0.0
            conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
            updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
            print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
        else:
          None
      for lista_atomi in fragments_dict.values():
        if atomo_finale == lista_atomi[0]:
          print(f'atomo finale da cui trovare il referee: {atomo_finale}')
          atomo_referee= trova_chiave(fragments_dict, lista_atomi)
          print(f'atomo referee: {atomo_referee}')
          coords_x_y_Finale= atomcoords_info_cdxml_tupla[atomo_referee]
          print(f'le coordinate dell\'atomo finale sono: {coords_x_y_Finale}')
          atom_idx_rdkit_da_cambiare=trova_chiave(dict_comparative_idx, atomo_finale)
          if atom_idx_rdkit_da_cambiare is None:
            print('non trovato')
          else:
            print(f'atom_idx_rdkit_da_cambiare: {atom_idx_rdkit_da_cambiare}')
            coord_dict_scaled[atom_idx_rdkit_da_cambiare]=coords_x_y_Finale
            print(f'dizionario aggiornato{coord_dict_scaled[atom_idx_rdkit_da_cambiare]}')
            conf = molobj.GetConformer()
            new_x, new_y = coords_x_y_Finale
            new_x=new_x/rapporto_x
            new_y=new_y/rapporto_y
            new_z = 0.0
            conf.SetAtomPosition(atom_idx_rdkit_da_cambiare, (new_x, new_y, new_z))
            updated_pos = conf.GetAtomPosition(atom_idx_rdkit_da_cambiare)
            print(f'le nuove coordinate dell\'atomo {atom_idx_rdkit_da_cambiare}, in rdkit sono: {updated_pos}')
        else:
          None
  #Modifica della stereochimica se il legame è wedge
      atomo_iniziale_rdkit=trova_chiave(coord_dict_scaled, coords_x_y_Iniziale)
      print(f'atomo iniziale: {atomo_iniziale_rdkit}')
      atomo_finale_rdkit=trova_chiave(coord_dict_scaled, coords_x_y_Finale)
      print(f'atomo finale: {atomo_finale_rdkit}')
      if atomo_iniziale_rdkit is None or atomo_finale_rdkit is None:
        print(color_string(f'errore: atomo iniziale cdxml {atomo_iniziale}={atomo_iniziale_rdkit}',color='red'))
        print(color_string(f'errore: atomo finale cdxml {atomo_finale}={atomo_finale_rdkit}',color='red'))
      else:
        legami_stereo = molobj.GetBondBetweenAtoms(atomo_iniziale_rdkit,atomo_finale_rdkit)
        if legami_stereo is None:
          print(color_string(f'non ci sono legami stereo tra {atomo_iniziale_rdkit} e {atomo_finale_rdkit}',color='red'))
        else:
          print(color_string(f'ci sono legami stereo Wedge tra {atomo_iniziale_rdkit} e {atomo_finale_rdkit} imposto STEREOANY e BEGINWEDGE',color='magenta'))
          legami_stereo.SetStereo(Chem.rdchem.BondStereo.STEREOANY)
          legami_stereo.SetBondDir(Chem.rdchem.BondDir.BEGINWEDGE)
    else:
      None
  return

In [19]:
def delete_atom_with_warning_info(molobj,pathcdxml):
  conf = molobj.GetConformer(0) # conformero della molecola
  coord_dict= {} # dizionario con le coordinate rdkit e indice rdkit
  for atom in molobj.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx()) #estrae le coordinate rdkit
    x, y, _ = pos  #ignora la coordinata Z con '_'
    coord_dict[atom.GetIdx()] = (x, y) # salva le coordinate nel dizionario con l'indice atomico rispettivo
  tree = ET.parse(pathcdxml)
  root = tree.getroot()
  atoms = root.findall('.//n')
  #dizionario con indici cdxml e coordinate cdxml
  atomcoords_info_cdxml = {}
  for atom in atoms:
    atom_id = int(atom.get('id'))  # si ottiene l'id dell'atomo
    coords = atom.get('p')  # si ottengono le coordinate cdmxl
    atomcoords_info_cdxml[atom_id] = coords
  #convertire le coordinate del dizionario cdxml in una tupla con numeri float
  atomcoords_info_cdxml_tupla=convert_string_to_float_tuple(atomcoords_info_cdxml)
  #convertire le coordinate di rdkit in cdxml con il fattore di scala
  rapporto_x, rapporto_y=trova_fattore_di_scala(atomcoords_info_cdxml_tupla,coord_dict)
  coord_dict_scaled = {k: (round(v[0] * rapporto_x, 2), round(v[1] * rapporto_y, 2)) for k, v in coord_dict.items()}
  dict_2={} # dizionario di matching tra gli atomi rdkit e cdxml
  for key_coord_dict, value_coord_dict in coord_dict_scaled.items():
    for key_atomcoords_dict, value_atomcoords_dict in atomcoords_info_cdxml_tupla.items():
      if value_coord_dict == value_atomcoords_dict:
        dict_2[key_coord_dict] = key_atomcoords_dict
        break
  # Creazione di una copia della molecola da modificare
  mol_copy = Chem.RWMol(molobj)
  atomi_da_rimuovere = []
  for atom in atoms:
    atom_id = int(atom.get('id'))
    atom_warning= atom.get('Warning')
    if atom_warning == 'This atom is very close to another atom or bond.':
      print(f'atom {atom_id} ha un warning: {atom_warning}')
      for key_dict_2, value_dict_2 in dict_2.items():
        if atom_id==value_dict_2:
          atom_RDKit=key_dict_2
          print(color_string(f'atomo rdkit: {atom_RDKit}',color='green'))
          atom_obj_RDKit=molobj.GetAtomWithIdx(atom_RDKit)
          atomi_vicini= atom_obj_RDKit.GetNeighbors()
          print(color_string(f'atomi vicini: {atomi_vicini}',color='white'))
          if atomi_vicini:
            print('questo atomo non va cancellato')
          else:
            print('questo atomo va cancellato')
            atomi_da_rimuovere.append(atom_RDKit)
  # Rimuovere gli atomi individuati
  atomi_da_rimuovere.sort(reverse=True)  # Rimuoviamo gli atomi a partire da quello con l'indice più alto
  for atom_idx in atomi_da_rimuovere:
      mol_copy.RemoveAtom(atom_idx)
      print(color_string(f'Atomo {atom_idx} rimosso.',color='magenta'))
  return mol_copy

In [20]:
def reset_radical_for_dative_bonds(molobj):
  for bond in molobj.GetBonds(): # si itera su tutti i legami della molecola
    atom_begin = bond.GetBeginAtom() # salvo l'atomo iniziale
    atom_end = bond.GetEndAtom() # salvo l'atomo finale
    bond_type=bond.GetBondType() # salvo il tipo di legame
    if bond_type== Chem.BondType.DATIVE: # se dativo analizzo se gli atomi hanno elettroni radicalici
      print(color_string(f'il legame tra gli atomi {atom_begin.GetIdx()} e {atom_end.GetIdx()} è dativo', color='yellow'))
      atom_begin_radical= atom_begin.GetNumRadicalElectrons() # analizzo se gli atomi hanno elettroni radicalici
      atom_end_radical= atom_end.GetNumRadicalElectrons()
      print(f'numero elettroni spaiati radicali atomo begin {atom_begin.GetIdx()} : {atom_begin_radical}')
      print(f'numero elettroni spaiati radicali atomo end {atom_end.GetIdx()} : {atom_end_radical}')
      if atom_begin_radical != 0: # se ne hanno reimposto a zero
        print(color_string(f'l\'atomo iniziale: {atom_begin.GetIdx()} è radicalico, tolgo gli elettroni spaiati', color='green'))
        atom_begin.SetNumRadicalElectrons(0)
      if atom_end_radical != 0:
        print(color_string(f'l\'atomo finale: {atom_end.GetIdx()} è radicalico, tolgo gli elettroni spaiati', color='green'))
        atom_end.SetNumRadicalElectrons(0)

In [21]:
def distanza_euclidea(coord1, coord2):
    """Calcola la distanza euclidea tra due punti (x1, y1) e (x2, y2)."""
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [22]:
def calcola_angolo_di_legame(mol, idx_central, idx1, idx2):
    """
    Calcola l'angolo di legame tra tre atomi (idx1-central-idx2).
    param mol: Molecola RDKit.
    param idx_central: Indice dell'atomo centrale (es. il carbonio).
    param idx1: Indice del primo atomo vicino.
    param idx2: Indice del secondo atomo vicino.
    return: Angolo in gradi.
    """
    conf = mol.GetConformer()
    # Ottieni le coordinate degli atomi
    pos_central = np.array(conf.GetAtomPosition(idx_central))
    pos1 = np.array(conf.GetAtomPosition(idx1))
    pos2 = np.array(conf.GetAtomPosition(idx2))
    # Calcola i vettori relativi all'atomo centrale
    vec1 = pos1 - pos_central
    vec2 = pos2 - pos_central
    # Normalizza i vettori
    vec1_normalized = vec1 / np.linalg.norm(vec1)
    vec2_normalized = vec2 / np.linalg.norm(vec2)
    # Calcola l'angolo tra i due vettori usando il prodotto scalare
    dot_product = np.dot(vec1_normalized, vec2_normalized)
    angle_radians = np.arccos(dot_product)  # Angolo in radianti
    # Converti in gradi
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

In [23]:
def calcola_posizione_dummy(mol, idx_carbonio_chirale, idx_atomo_vicino_1, idx_atomo_vicino_2, d=0.3):
    """
    Calcola la posizione dell'atomo dummy (asterisco) vicino al carbonio chirale,
    posizionandolo lungo la bisettrice dell'angolo di legame tra due atomi vicini,
    all'interno del triangolo formato dal carbonio chirale e i due atomi vicini.

    Argomenti della funzione:
        mol: oggetto molecolare RDKit
        idx_carbonio_chirale: indice del carbonio chirale
        idx_atomo_vicino_1: indice del primo atomo vicino
        idx_atomo_vicino_2: indice del secondo atomo vicino
        d: distanza dalla bisettrice per posizionare l'atomo dummy
    Returns:
        (x_dummy, y_dummy): coordinate x e y del nuovo atomo dummy
    """
    # Ottieni il conformero della molecola (assumo ci sia almeno un conformero, il primo)
    conf = mol.GetConformer()
    # Estrai le coordinate degli atomi (solamente x, y)
    coord_carbonio_chirale = np.array([conf.GetAtomPosition(idx_carbonio_chirale).x, conf.GetAtomPosition(idx_carbonio_chirale).y])
    coord_atomo_vicino_1 = np.array([conf.GetAtomPosition(idx_atomo_vicino_1).x, conf.GetAtomPosition(idx_atomo_vicino_1).y])
    coord_atomo_vicino_2 = np.array([conf.GetAtomPosition(idx_atomo_vicino_2).x, conf.GetAtomPosition(idx_atomo_vicino_2).y])
    # Calcola i vettori diretti dai due atomi vicini verso il carbonio chirale
    vA = coord_carbonio_chirale - coord_atomo_vicino_1
    vB = coord_carbonio_chirale - coord_atomo_vicino_2
    # Somma dei vettori per trovare la direzione della bisettrice
    vSum = vA + vB
    # Normalizza il vettore della bisettrice
    vSum_normalized = vSum / np.linalg.norm(vSum)
    # Calcola il punto medio tra i due atomi vicini
    punto_medio = (coord_atomo_vicino_1 + coord_atomo_vicino_2) / 2
    # Condizione per determinare la direzione (sinistra o destra rispetto al carbonio chirale)
    if punto_medio[0] < coord_carbonio_chirale[0]:
        # La bisettrice è orientata a sinistra
        new_coords_dummy = coord_carbonio_chirale - vSum_normalized * d
    else:
        # La bisettrice è orientata a destra
        new_coords_dummy = coord_carbonio_chirale + vSum_normalized * d
    # Restituisci le nuove coordinate x, y
    return new_coords_dummy[0], new_coords_dummy[1]

In [24]:
def add_label(molobj, pathcdxml, verboso=False):
  conf = molobj.GetConformer(0) # conformero della molecola
  coord_dict= {} # dizionario con le coordinate rdkit e indice rdkit
  for atom in molobj.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx()) #estrae le coordinate rdkit
    x, y, _ = pos  #ignora la coordinata Z con '_'
    # salva le coordinate nel dizionario con l'indice atomico rispettivo
    coord_dict[atom.GetIdx()] = (x, y)
  tree = ET.parse(pathcdxml)
  root = tree.getroot()
  atoms = root.findall('.//n')
  #dizionario con indici cdxml e coordinate cdxml
  atomcoords_info_cdxml = {}
  for atom in atoms:
    atom_id = int(atom.get('id'))  # si ottiene l'id dell'atomo
    coords = atom.get('p')  # si ottengono le coordinate cdmxl
    atomcoords_info_cdxml[atom_id] = coords
  #convertire le coordinate del dizionario cdxml in una tupla con numeri float
  atomcoords_info_cdxml_tupla=convert_string_to_float_tuple(atomcoords_info_cdxml)
  #convertire le coordinate di rdkit in cdxml con il fattore di scala
  rapporto_x, rapporto_y=trova_fattore_di_scala(atomcoords_info_cdxml_tupla,coord_dict)
  coord_dict_scaled = {k: (round(v[0] * rapporto_x, 2), round(v[1] * rapporto_y, 2)) for k, v in coord_dict.items()}
  dict_2={} # dizionario di matching tra gli atomi rdkit e cdxml
  for key_coord_dict, value_coord_dict in coord_dict_scaled.items():
    for key_atomcoords_dict, value_atomcoords_dict in atomcoords_info_cdxml_tupla.items():
      if value_coord_dict == value_atomcoords_dict:
        dict_2[key_coord_dict] = key_atomcoords_dict
        break
  molobj_copia = Chem.RWMol(molobj) # creo una copia della molecola
  texts = root.findall('.//t') # si cerca nel file cdxml le stringhe di testo
  graphic= root.findall('.//graphic') # si cercano nel file cdxml elementi grafici
  for text in texts: #iterazione sulle stringhe
    id = int(text.get('id')) # id delle stringhe
    coords = text.get('p') # coordinate delle stringhe
    x, y = coords.split()
    x = float(x)
    y = float(y)
    atom_coords = (x, y) # coordinate cdxml convertite in tupla con numeri float
    if verboso:
      print(color_string(f'id: {id}', color='yellow'))
    label = root.find(f".//t[@id='{id}']/s") # dalla stringa ricavo l'etichetta
    if label is not None: # se presente
      stringa = label.text.strip()  # Rimuove spazi vuoti e newline iniziali/finali
      # normalizzazione della stringa
      normalized_stringa = " ".join(unicodedata.normalize("NFKC", stringa.replace("\n", " ")).split())
      print(color_string(f'stringa da aggiungere = {normalized_stringa}', color='green'))
      # negli if ed elif a seguire
      # Controlliamo la stringa normalizzata se presenta il testo di interesse da aggiungere
      if normalized_stringa in ["and epimer at C* et l'épimère en C* y el epímero al C*",
                                "and epimer at C* et l'épimère en C* y el epímero en C*"] :
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        conformer = molobj_copia.GetConformer(0)  # conformero della copia
        # Creiamo un atomo dummy ogni riga
        label_line1 = "and epimer at C*" # reale riferimento per draw_OMS_molecule
        label_line2 = "                " # spazio vuoto
        label_line3 = "                " # spazio vuoto
        offset_y = 0.5  # Spostamento verticale tra le righe
        # si ricavano le coordinate che avrà l'atomo dummy per la riga 1
        new_x, new_y, new_z = atom_coords[0] / rapporto_x, atom_coords[1] / rapporto_y, 0.0
        # creazione e aggiunta del primo atomo dummy
        dummy_atom1 = Chem.Atom('*')
        atom_idx1 = molobj_copia.AddAtom(dummy_atom1)
        molobj_copia.GetAtomWithIdx(atom_idx1).SetProp('atomLabel', label_line1)
        conformer.SetAtomPosition(atom_idx1, (new_x, new_y, new_z))
        # Aggiunta del secondo atomo dummy con uno spostamento in Y
        dummy_atom2 = Chem.Atom('*')
        atom_idx2 = molobj_copia.AddAtom(dummy_atom2)
        molobj_copia.GetAtomWithIdx(atom_idx2).SetProp('atomLabel', label_line2)
        conformer.SetAtomPosition(atom_idx2, (new_x, new_y - offset_y, new_z))
        # Aggiunta del terzo atomo dummy con un ulteriore spostamento in Y
        dummy_atom3 = Chem.Atom('*')
        atom_idx3 = molobj_copia.AddAtom(dummy_atom3)
        molobj_copia.GetAtomWithIdx(atom_idx3).SetProp('atomLabel', label_line3)
        conformer.SetAtomPosition(atom_idx3, (new_x, new_y - 2 * offset_y, new_z))
      elif normalized_stringa == "and epimer at P et l'épimère en P y el epímero al P" :
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        conformer = molobj_copia.GetConformer(0)  # conformero della copia
        # Creiamo un atomo dummy per ogni riga
        label_line1 = "and epimer at P" # reale riferimento per draw_OMS_molecule
        label_line2 = "                " # spazio vuoto
        label_line3 = "                " # spazio vuoto
        offset_y = 0.5  # Spostamento verticale tra le righe
        # si ricavano le coordinate che avrà l'atomo dummy per la riga 1
        new_x, new_y, new_z = atom_coords[0] / rapporto_x, atom_coords[1] / rapporto_y, 0.0
        # creazione e aggiunta del primo atomo dummy
        dummy_atom1 = Chem.Atom('*')
        atom_idx1 = molobj_copia.AddAtom(dummy_atom1)
        molobj_copia.GetAtomWithIdx(atom_idx1).SetProp('atomLabel', label_line1)
        conformer.SetAtomPosition(atom_idx1, (new_x, new_y, new_z))
        # Aggiunta del secondo atomo dummy con uno spostamento in Y
        dummy_atom2 = Chem.Atom('*')
        atom_idx2 = molobj_copia.AddAtom(dummy_atom2)
        molobj_copia.GetAtomWithIdx(atom_idx2).SetProp('atomLabel', label_line2)
        conformer.SetAtomPosition(atom_idx2, (new_x, new_y - offset_y, new_z))
        # Aggiunta del terzo atomo dummy con un ulteriore spostamento in Y
        dummy_atom3 = Chem.Atom('*')
        atom_idx3 = molobj_copia.AddAtom(dummy_atom3)
        molobj_copia.GetAtomWithIdx(atom_idx3).SetProp('atomLabel', label_line3)
        conformer.SetAtomPosition(atom_idx3, (new_x, new_y - 2 * offset_y, new_z))
      elif normalized_stringa in ["and enantiomer et énantiomère y enantiómero",
                                  "and enantiomer et énantiomère and enantiómero",
                                  "and enantiomer et énantiomère y entantiómero",
                                  "and enantiomer et énantiomère y el enantiómero"]:
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        conformer = molobj_copia.GetConformer(0)  # conformero della copia
        label_line1 = "and enantiomer" # reale riferimento per draw_OMS_molecule
        label_line2 = "                " # spazio vuoto
        label_line3 = "                " # spazio vuoto
        offset_y = 0.5  # Spostamento verticale tra le righe
        # si ricavano le coordinate che avrà l'atomo dummy per la riga 1
        new_x, new_y, new_z = atom_coords[0] / rapporto_x, atom_coords[1] / rapporto_y, 0.0
        # creazione e aggiunta del primo atomo dummy
        dummy_atom1 = Chem.Atom('*')
        atom_idx1 = molobj_copia.AddAtom(dummy_atom1)
        molobj_copia.GetAtomWithIdx(atom_idx1).SetProp('atomLabel', label_line1)
        conformer.SetAtomPosition(atom_idx1, (new_x, new_y, new_z))
        # Aggiungi il secondo atomo dummy con uno spostamento in Y
        dummy_atom2 = Chem.Atom('*')
        atom_idx2 = molobj_copia.AddAtom(dummy_atom2)
        molobj_copia.GetAtomWithIdx(atom_idx2).SetProp('atomLabel', label_line2)
        conformer.SetAtomPosition(atom_idx2, (new_x, new_y - offset_y, new_z))
        # Aggiungi il terzo atomo dummy con un ulteriore spostamento in Y
        dummy_atom3 = Chem.Atom('*')
        atom_idx3 = molobj_copia.AddAtom(dummy_atom3)
        molobj_copia.GetAtomWithIdx(atom_idx3).SetProp('atomLabel', label_line3)
        conformer.SetAtomPosition(atom_idx3, (new_x, new_y - 2 * offset_y, new_z))
      elif normalized_stringa == "* and enantiomer * et énantiomère * y enantiómero":
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        conformer = molobj_copia.GetConformer(0)  # conformero della copia
        # Creiamo tre atomi dummy per le tre righe
        label_line1 = "* and enantiomer" # reale riferimento per draw_OMS_molecule
        label_line2 = "                " # spazio vuoto
        label_line3 = "                " # spazio vuoto
        offset_y = 0.5  # Spostamento verticale tra le righe
        # si ricavano le coordinate che avrà l'atomo dummy per la riga 1
        new_x, new_y, new_z = atom_coords[0] / rapporto_x, atom_coords[1] / rapporto_y, 0.0
        # creazione e aggiunta del primo atomo dummy
        dummy_atom1 = Chem.Atom('*')
        atom_idx1 = molobj_copia.AddAtom(dummy_atom1)
        molobj_copia.GetAtomWithIdx(atom_idx1).SetProp('atomLabel', label_line1)
        conformer.SetAtomPosition(atom_idx1, (new_x, new_y, new_z))
        # Aggiungi il secondo atomo dummy con uno spostamento in Y
        dummy_atom2 = Chem.Atom('*')
        atom_idx2 = molobj_copia.AddAtom(dummy_atom2)
        molobj_copia.GetAtomWithIdx(atom_idx2).SetProp('atomLabel', label_line2)
        conformer.SetAtomPosition(atom_idx2, (new_x, new_y - offset_y, new_z))
        # Aggiungi il terzo atomo dummy con un ulteriore spostamento in Y
        dummy_atom3 = Chem.Atom('*')
        atom_idx3 = molobj_copia.AddAtom(dummy_atom3)
        molobj_copia.GetAtomWithIdx(atom_idx3).SetProp('atomLabel', label_line3)
        conformer.SetAtomPosition(atom_idx3, (new_x, new_y - 2 * offset_y, new_z))
      elif normalized_stringa == "or H":
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        conformer = molobj_copia.GetConformer(0)  # conformero della copia
        label_line1 = "or H<sup>+</sup>" # reale riferimento per draw_OMS_molecule
        new_x, new_y, new_z = atom_coords[0] / rapporto_x, atom_coords[1] / rapporto_y, 0.0
        dummy_atom1 = Chem.Atom('*')
        atom_idx1 = molobj_copia.AddAtom(dummy_atom1)
        molobj_copia.GetAtomWithIdx(atom_idx1).SetProp('atomLabel', label_line1)
        conformer.SetAtomPosition(atom_idx1, (new_x, new_y, new_z))
      elif normalized_stringa == "-":
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        conformer = molobj_copia.GetConformer(0)  # conformero della copia
        label_line1 = "-" # reale riferimento per draw_OMS_molecule
        new_x, new_y, new_z = atom_coords[0] / rapporto_x, atom_coords[1] / rapporto_y, 0.0
        dummy_atom1 = Chem.Atom('*')
        atom_idx1 = molobj_copia.AddAtom(dummy_atom1)
        molobj_copia.GetAtomWithIdx(atom_idx1).SetProp('atomLabel', label_line1)
        conformer.SetAtomPosition(atom_idx1, (new_x, new_y, new_z))
      elif normalized_stringa== "*":
        print(color_string(f'stringa aggiunta = {normalized_stringa}', color='red'))
        # creazione e aggiunta dell'atomo dummy asterisco
        dummy_atom = Chem.Atom('*')
        atom_idx = molobj_copia.AddAtom(dummy_atom)
        molobj_copia.GetAtomWithIdx(atom_idx).SetProp('atomLabel', '*')
        New_atom_coords_x , New_atom_coords_y = atom_coords  # Ottiene le coordinate
        New_x= New_atom_coords_x/rapporto_x # coordinata x convertite in rdkit
        New_y= New_atom_coords_y/rapporto_y # coordinata y convertite in rdkit
        New_z = 0.0
        dummy_coords= (New_x,New_y)
        print(color_string(f'le nuove coordinate CDXML dell\'* sono: {dummy_coords}', color='red'))
        # dizionario con indici atomici degli atomi chirali e distanza di questi dall'atomo dummy
        distance_atoms_chiral={}
        for atom in molobj_copia.GetAtoms(): # itera su ogni atomo della molecola
          atom_id=atom.GetIdx()
          if verboso:
            print(color_string(f'indice atomo: {atom_id}', color='white'))
          atom_element=atom.GetAtomicNum() # ricava il numero atomico
          atom_chirality=atom.GetChiralTag() # ricava la chiralità
          if atom_element != 0:
            if verboso:
              print(color_string(f'numero atomico atomo: {atom_element}', color='white'))
            # se l'atomo è chirale
            if atom_chirality == Chem.ChiralType.CHI_TETRAHEDRAL_CCW or atom_chirality== Chem.ChiralType.CHI_TETRAHEDRAL_CW:
              if verboso:
                print(color_string(f'tag chiralità atomo: {atom_chirality}', color='white'))
              pos = conf.GetAtomPosition(atom.GetIdx()) #estrae le coordinate rdkit
              x, y, _ = pos  #ignora la coordinata Z con '_'
              coord_chiral= (x,y) # coordinate salvate come tupla
              print(color_string(f'le coordinate di {atom_id} sono: {coord_chiral}', color='green'))
              # calcolo della distanza euclidea
              distanza_euclidea_chiral_atom=distanza_euclidea(dummy_coords,coord_chiral)
              print(color_string(f'la distanza euclidea tra l\'* e l\'atomo {atom_id} è: {distanza_euclidea_chiral_atom}', color='green'))
              # aggiunta della distanza euclidea e l'atomo chirale rispettivo al dizionario
              distance_atoms_chiral[atom_id]=distanza_euclidea_chiral_atom
              # si ordina il dizionario in maniera crescente rispetto le distanze
              distance_atoms_chiral_sorted = dict(sorted(distance_atoms_chiral.items(), key=lambda item: item[1]))
        if distance_atoms_chiral is not None:
          # dal primo elemento del dizionario ricava l'atomo chirale più vicino
          atomo_più_vicino = list(distance_atoms_chiral_sorted.keys())[0]
          distanza_euclidea_chiral_atom = distance_atoms_chiral_sorted[atomo_più_vicino]
          print(color_string(f'l\'atomo più vicino è: {atomo_più_vicino}', color='green'))
          print(color_string(f'la distanza euclidea è: {distance_atoms_chiral_sorted[atomo_più_vicino]}', color='yellow'))
          # si procede se la distanza euclidea è minore di 2.0001
          if distanza_euclidea_chiral_atom < 2.0001:
            # trova gli atomi vicini al centro chirale
            atom_neighbours = [neighbor.GetIdx() for neighbor in molobj_copia.GetAtomWithIdx(atomo_più_vicino).GetNeighbors()]
            print(color_string(f'i vicini dell\'atomo più vicino sono: {atom_neighbours}', color='green'))
            atom_neighbours_coords={}
            for atom_neighbour in atom_neighbours: # itera sui vicini
              pos = conf.GetAtomPosition(atom_neighbour) #estrae le coordinate rdkit
              x, y, _ = pos  #ignora la coordinata Z con '_'
              coord_neighbour= (x,y) # coordinate come tupla
              # aggiorna il dizionario con gli indici dei vicini e le coordinate
              atom_neighbours_coords[atom_neighbour]=coord_neighbour
            if len(atom_neighbours_coords)==4: # si procede se i vicini all'atomo chirale sono 4
              # calcolo degli atomi di legame
              angoli_e_atomi = [] # lista che conterrà l'angolo di legame e i due atomi vicini che lo formano
              angolo_1 = calcola_angolo_di_legame(molobj_copia, atomo_più_vicino, atom_neighbours[0], atom_neighbours[1])
              angoli_e_atomi.append((angolo_1, atom_neighbours[0], atom_neighbours[1]))
              angolo_2 = calcola_angolo_di_legame(molobj_copia, atomo_più_vicino, atom_neighbours[1], atom_neighbours[2])
              angoli_e_atomi.append((angolo_2, atom_neighbours[1], atom_neighbours[2]))
              angolo_3 = calcola_angolo_di_legame(molobj_copia, atomo_più_vicino, atom_neighbours[2], atom_neighbours[3])
              angoli_e_atomi.append((angolo_3, atom_neighbours[2], atom_neighbours[3]))
              angolo_4 = calcola_angolo_di_legame(molobj_copia, atomo_più_vicino, atom_neighbours[3], atom_neighbours[0])
              angoli_e_atomi.append((angolo_4, atom_neighbours[3], atom_neighbours[0]))
              print(color_string(f'angolo 1: {angolo_1}', color='green'))
              print(color_string(f'angolo 2: {angolo_2}', color='green'))
              print(color_string(f'angolo 3: {angolo_3}', color='green'))
              print(color_string(f'angolo 4: {angolo_4}', color='green'))
              somma_angolo = angolo_1 + angolo_2 + angolo_3 + angolo_4
              print(color_string(f'somma angoli: {somma_angolo}', color='green'))
              # si procede se gli angoli calcolati danno sommma 360 gradi
              if somma_angolo == 360:
                  print(color_string(f'somma angoli corretta: 360°', color='green'))
                  # Trova l'angolo di legame più grande e i rispettivi atomi vicini
                  angolo_massimo, atomo1, atomo2 = max(angoli_e_atomi, key=lambda x: x[0])
                  print(color_string(f'Angolo di legame massimo: {angolo_massimo}', color='blue'))
                  print(color_string(f'Atomi vicini con angolo massimo: {atomo1} - {atomo2}', color='blue'))
                  # si calcola la posizione dell'asterisco rispetto l'angolo maggiore
                  coord_dummy_x,coord_dummy_y=calcola_posizione_dummy(molobj_copia,atomo_più_vicino,atomo1,atomo2)
                  print(color_string(f'le nuove coordinate dell\'* sono: {coord_dummy_x} {coord_dummy_y}', color='red'))
                  conformer = molobj_copia.GetConformer(0)  # conformero della copia
                  # Imposta le nuove coordinate per l'atomo aggiunto
                  conformer.SetAtomPosition(atom_idx, (coord_dummy_x,coord_dummy_y,New_z))
              else: # se la somma angoli è differente
                  # le coordinate rimarranno quelle scritte nel file cdxml
                  print(color_string(f'somma angoli sbagliata', color='green'))
                  conformer = molobj_copia.GetConformer(0)  # conformero della copia
                  conformer.SetAtomPosition(atom_idx, (New_x,New_y,New_z))
            else: # se l'atomo chirale non lega 4 atomi
              # le coordinate rimarranno quelle scritte nel file cdxml
              print(color_string(f'atomi vicini non sono 4', color='red'))
              conformer = molobj_copia.GetConformer(0)  # conformero della copia
              conformer.SetAtomPosition(atom_idx, (New_x,New_y,New_z))
          else: # se la distanza euclidea dovesse essere maggiore,
                # si posiziona di fianco l'atomo non chirale più vicino
            # dizionario con indici atomici degli atomi non chirali e distanza di questi dall'atomo dummy
            distance_atoms_not_chiral={} # inizializzazione dizionario
            print(color_string(f'distanza euclidea troppo elevata {distanza_euclidea_chiral_atom}', color='yellow'))
            for atom in molobj_copia.GetAtoms():
              atom_id=atom.GetIdx()
              atom_neighbours = [neighbor.GetIdx() for neighbor in molobj_copia.GetAtomWithIdx(atom_id).GetNeighbors()]
              if len(atom_neighbours)==3: # se gli atomi vicini sono 3
                pos = conf.GetAtomPosition(atom.GetIdx()) #estrae le coordinate rdkit
                x, y, _ = pos  #ignora la coordinata Z con '_'
                coord_not_chiral= (x,y)
                distanza_euclidea_not_chiral_atom=distanza_euclidea(dummy_coords,coord_not_chiral)
                distance_atoms_not_chiral[atom_id]=distanza_euclidea_not_chiral_atom
                distance_atoms_not_chiral_sorted = dict(sorted(distance_atoms_not_chiral.items(), key=lambda item: item[1]))
            if distance_atoms_not_chiral:
              atomo_più_vicino = list(distance_atoms_not_chiral_sorted.keys())[0]
              atomo_più_vicino_coords= conf.GetAtomPosition(atomo_più_vicino)
              print(color_string(f'l\'atomo più vicino è: {atomo_più_vicino}', color='green'))
              x_near,y_near, _=atomo_più_vicino_coords
              print(color_string(f'le coordinate del più vicino sono: {x_near} {y_near}', color='green'))
              distanza_euclidea_not_chiral_atom = distance_atoms_not_chiral_sorted[atomo_più_vicino]
              if distanza_euclidea_not_chiral_atom < 2.0001:
                print(color_string(f'distanza euclidea corretta: {distanza_euclidea_not_chiral_atom}', color='green'))
                atom_neighbours = [neighbor.GetIdx() for neighbor in molobj_copia.GetAtomWithIdx(atomo_più_vicino).GetNeighbors()]
                print(color_string(f'i vicini dell\'atomo più vicino sono: {atom_neighbours}', color='green'))
                sum_y_dict={} # differenze coordinate y
                sum_x_dict={} # differenze coordinate x
                atom_neighbours_coords={} # dizionario delle coordinate degli atomi vicini
                for atom_neighbour in atom_neighbours:
                  pos = conf.GetAtomPosition(atom_neighbour) #estrae le coordinate rdkit
                  x, y, _ = pos  #ignora la coordinata Z con '_'
                  coord_neighbour= (x,y)
                  sum_y= abs(y-y_near)
                  sum_x= x-x_near
                  print(f'l\'atomo vicino: {atom_neighbour} ha somma x: {sum_x} e somma y: {sum_y} ' )
                  sum_x_dict[sum_x]=atom_neighbour
                  sum_y_dict[sum_y]=atom_neighbour
                  atom_neighbours_coords[atom_neighbour]=coord_neighbour
                min_sum_y = min(sum_y_dict.keys())  # Trova il valore minimo di sum_y
                atom_neighbour_min_y = sum_y_dict[min_sum_y]  # Ottieni l'atom_neighbour corrispondente
                print(f'l\'atomo con somma y minore è: {atom_neighbour_min_y}')
                # ottiene le coordinate dell'atomo con differenza y minore
                coord_x_atom_neighbour_min_y=atom_neighbours_coords[atom_neighbour_min_y][0]
                coord_y_atom_neighbour_min_y=atom_neighbours_coords[atom_neighbour_min_y][1]
                m=(coord_y_atom_neighbour_min_y-y_near)/(coord_x_atom_neighbour_min_y-x_near)
                print(f'la m è: {m}')
                angolo= abs(math.atan(m))
                angolo_in_gradi= math.degrees(angolo)
                print(f'l\'angolo è: {angolo_in_gradi}')
                if coord_x_atom_neighbour_min_y-x_near < 0:
                  distanza_da_near=  0.3
                else:
                  distanza_da_near= - 0.3
                dx = math.cos(angolo) * distanza_da_near
                dy = math.sin(angolo) * distanza_da_near
                New_x= x_near + dx
                New_y= y_near + dy
                New_z = 0.0
                print(color_string(f'le nuove coordinate dell\'* sono: {New_x} {New_y}', color='red'))
                conformer = molobj_copia.GetConformer(0)  # conformero della copia
                conformer.SetAtomPosition(atom_idx, (New_x,New_y,New_z))
        else:
          print(color_string(f'Nessun atomo chirale trovato', color='red'))
          conformer = molobj_copia.GetConformer(0)  # conformero della copia
          conformer.SetAtomPosition(atom_idx, (New_x,New_y,New_z))
      else:
        print("Tag <s> non trovato.")
    else:
        print("Label is none, Tag <s> non trovato.")
  # iterazione sugli elementi grafici per aggiungere eventuali parentesi quadre
  for graph in graphic:
    id = int(graph.get('id'))
    print(color_string(f'graphic_id: {id}', color='yellow'))
    coords = graph.get('BoundingBox') # BoundingBox utile ad ottenere le coordinate
    graphicType= graph.get('GraphicType')
    if coords:
      x1, y1, x2, y2 = coords.split()
      x1 = float(x1)
      y1 = float(y1)
      x2 = float(x2)
      y2 = float(y2)
      atom_coords_1 = (x1, y1)
      atom_coords_2 = (x2, y2)
      print(color_string(f'le coordinate del primo atomo sono: {atom_coords_1}', color='green'))
      print(color_string(f'le coordinate del secondo atomo sono: {atom_coords_2}', color='green'))
      if graphicType=="Bracket": # se ci sono parentesi quadre
      # aggiunta di atomi dummy rappresentanti i vertici
        dummy_1= Chem.Atom('*')
        dummy_2= Chem.Atom('*')
        atom_idx_1 = molobj_copia.AddAtom(dummy_1)
        atom_idx_2 = molobj_copia.AddAtom(dummy_2)
        molobj_copia.GetAtomWithIdx(atom_idx_1).SetProp('atomLabel', '|')
        molobj_copia.GetAtomWithIdx(atom_idx_2).SetProp('atomLabel', '|')
        new_x1, new_y1, new_z1 = atom_coords_1[0]/rapporto_x, atom_coords_1[1]/rapporto_y, 0.0
        new_x2, new_y2, new_z2 = atom_coords_2[0]/rapporto_x, atom_coords_2[1]/rapporto_y, 0.0
        conformer = molobj_copia.GetConformer(0)
        conformer.SetAtomPosition(atom_idx_1, (new_x1, new_y1, new_z1))
        conformer.SetAtomPosition(atom_idx_2, (new_x2, new_y2, new_z2))
        print(color_string(f'le nuove coordinate del primo atomo sono: {new_x1} {new_y1}', color='green'))
        print(color_string(f'le nuove coordinate del secondo atomo sono: {new_x2} {new_y2}', color='green'))
  return molobj_copia

In [25]:
def draw_OMS_molecule(molecola, pathcdxml, nome_file, verboso=False):
    conf = molecola.GetConformer(0) # conformero della molecola
    coord_dict= {} # dizionario con le coordinate rdkit e indice rdkit
    for atom in molecola.GetAtoms():
      pos = conf.GetAtomPosition(atom.GetIdx()) #estrae le coordinate rdkit
      x, y, _ = pos  #ignora la coordinata Z con '_'
      # salva le coordinate nel dizionario con l'indice atomico rispettivo
      coord_dict[atom.GetIdx()] = (x, y)
    tree = ET.parse(pathcdxml)
    root = tree.getroot()
    atoms = root.findall('.//n')
    #dizionario con indici cdxml e coordinate cdxml
    atomcoords_info_cdxml = {}
    for atom in atoms:
      atom_id = int(atom.get('id'))  # si ottiene l'id dell'atomo
      coords = atom.get('p')  # si ottengono le coordinate cdmxl
      atomcoords_info_cdxml[atom_id] = coords
    #convertire le coordinate del dizionario cdxml in una tupla con numeri float
    atomcoords_info_cdxml_tupla=convert_string_to_float_tuple(atomcoords_info_cdxml)
    #convertire le coordinate di rdkit in cdxml con il fattore di scala
    rapporto_x, rapporto_y=trova_fattore_di_scala(atomcoords_info_cdxml_tupla,coord_dict)
    coord_dict_scaled = {k: (round(v[0] * rapporto_x, 2), round(v[1] * rapporto_y, 2)) for k, v in coord_dict.items()}
    dict_2={} # dizionario di matching tra gli atomi rdkit e cdxml
    for key_coord_dict, value_coord_dict in coord_dict_scaled.items():
      for key_atomcoords_dict, value_atomcoords_dict in atomcoords_info_cdxml_tupla.items():
        if value_coord_dict == value_atomcoords_dict:
          dict_2[key_coord_dict] = key_atomcoords_dict
          break
    if verboso:
      for key, value in dict_2.items():
        print(f'idx RDKit: {key} idx CDXML: {value}')
    # Crea un oggetto MolDraw2D per disegnare la molecola
    def imposta_opzioni(drawer):
        opzioni = drawer.drawOptions() # inizializzazione delle opzioni
        opzioni.flagCloseContactsDist = -1 # visualizzazione di claim per sovrapposizioni
        opzioni.explicitMethyl = True # esplicitazione dei metili terminali
        opzioni.useMolAbbreviations = True # applicazione delle abbreviazioni
        opzioni.addAtomIndices = False # indici atomici
        opzioni.addBondIndices = False # indici di legami
        opzioni.kekulize = True # kekulizzazione
        opzioni.useBWAtomPalette()  # Tavolozza monocromatica
        return opzioni
    # Crea il disegnatore
    drawer = rdMolDraw2D.MolDraw2DCairo(1080, 600)
    opzioni = imposta_opzioni(drawer)
    # Imposta un dizionario per nascondere la carica formale per atomi specifici
    nascondi_carica = {}
    # Itera sui legami per gestire l'aromaticità e il tipo di legame
    for bond in molecola.GetBonds():
        if bond.GetBondDir() == Chem.BondDir.BEGINDASH:
            bond.SetBondType(Chem.BondType.ZERO)
            bond.SetProp("_MolFileBondType", "0")
            atom_begin = bond.GetBeginAtom().GetIsAromatic()
            atom_end = bond.GetEndAtom().GetIsAromatic()
            atom_begin_symbol = bond.GetBeginAtom().GetSymbol()
            atom_end_symbol = bond.GetEndAtom().GetSymbol()
            atom_begin_formal_charge = bond.GetBeginAtom().GetFormalCharge()
            atom_end_formal_charge = bond.GetEndAtom().GetFormalCharge()
            atom_b=bond.GetBeginAtom()
            atom_e=bond.GetEndAtom()
            if atom_begin or atom_end:
                Chem.SanitizeMol(molecola, sanitizeOps=Chem.SanitizeFlags.SANITIZE_SETAROMATICITY)
            if atom_begin_symbol != 'C' and atom_begin and atom_begin_formal_charge==0:
                atom_b.SetFormalCharge(-1)  # Imposta una carica formale positiva sull'azoto
                nascondi_carica[atom_b.GetIdx()] = True  # Nascondi la carica di questo atomo
            if atom_end_symbol != 'C' and atom_end and atom_end_formal_charge==0:
                atom_e.SetFormalCharge(-1)  # Imposta una carica formale positiva sull'azoto
                nascondi_carica[atom_e.GetIdx()] = True  # Nascondi la carica di questo atomo
    # Personalizza le opzioni di disegno per nascondere la carica formale di atomi specifici
    for idx, atom in enumerate(molecola.GetAtoms()):
        if nascondi_carica.get(atom.GetIdx(), False):
          # Mostra solo il simbolo senza carica formale
            opzioni.atomLabels[idx] = atom.GetSymbol()
    # Disegna la molecola la prima volta
    rdMolDraw2D.PrepareAndDrawMolecule(drawer, molecola)
    # Raccogli le coordinate delle etichette
    etichette_coordinate = []
    for atom in molecola.GetAtoms():
        atom_idx = atom.GetIdx()
        coordinate = drawer.GetDrawCoords(atom.GetIdx())
        etichette_coordinate.append((atom.GetSymbol(), coordinate))
    # Controlla la presenza di sovrapposizioni
    sovrapposizioni = False
    soglia_sovrapposizione = 22  # Soglia di sovrapposizione
    for i in range(len(etichette_coordinate)):
        for j in range(i + 1, len(etichette_coordinate)):
            etichetta1, coord1 = etichette_coordinate[i]
            etichetta2, coord2 = etichette_coordinate[j]
            distanza = np.linalg.norm(np.array(coord1) - np.array(coord2))
            if distanza < soglia_sovrapposizione:
                sovrapposizioni = True
                print(f"Sovrapposizione tra '{etichetta1}' e '{etichetta2}'")
                break  # Esci dal ciclo interno se c'è una sovrapposizione
        if sovrapposizioni:
            print(f"Sovrapposizione trovata per, ridimensiono il font")
            # Ridisegna con un font più piccolo
            drawer = rdMolDraw2D.MolDraw2DCairo(1080, 600)
            opzioni = imposta_opzioni(drawer)
            opzioni.baseFontSize = 0.45  # Riduce la dimensione del font
            rdMolDraw2D.PrepareAndDrawMolecule(drawer, molecola)
            break  # Evita di continuare il loop esterno
    label_atom_coords = {}
    atomi_da_cancellare= []
    for atom in molecola.GetAtoms():
        atom_idx = atom.GetIdx()
        coordinate = drawer.GetDrawCoords(atom.GetIdx())
        coordinata_x= coordinate.x
        coordinata_y= coordinate.y
        if atom.HasProp('atomLabel'):  # Controlla se esiste la proprietà 'atomLabel'
          label = atom.GetProp('atomLabel')  # Ottieni la proprietà 'atomLabel'
          print(f"Atomo {atom_idx} ha l'etichetta: {label}")
          if label in ["and epimer at C*","and epimer at P","and enantiomer","* and enantiomer","or H<sup>+</sup>","|","-"]:
            atomi_da_cancellare.append(atom_idx)
            if label in label_atom_coords:
                label_atom_coords[label].append((coordinata_x, coordinata_y))
            else:
                # Se il label non esiste, lo inizializziamo come lista contenente la prima coppia di coordinate
                label_atom_coords[label] = [(coordinata_x, coordinata_y)]
    if label_atom_coords:
      print(f'label_atom_coords {label_atom_coords}')
    if atomi_da_cancellare:
      print(f'atomi_da_cancellare {atomi_da_cancellare}')
      for atom_idx in atomi_da_cancellare:
        atom_idx_rdkit=molecola.GetAtomWithIdx(atom_idx)
        label=atom_idx_rdkit.GetProp('atomLabel')
        if label=='|' or label=='-':
          atom_idx_rdkit.SetProp('atomLabel', ' ')
          print(color_string(f'Atomo linea o meno {atom_idx} rimosso.',color='red'))
        else:
          atom_idx_rdkit.SetProp('atomLabel', '                 ')
          print(color_string(f'Atomo stringa {atom_idx} rimosso.',color='red'))
    # Completa il disegno e salva
    drawer.ClearDrawing()
    rdMolDraw2D.PrepareAndDrawMolecule(drawer, molecola)
    drawer.FinishDrawing()
    drawer.WriteDrawingText(nome_file)
    if label_atom_coords: # se ci sono etichette da aggiungere
      numero= None # inizializzo la variabile numero
      img = plt.imread(nome_file) # lettura del file PNG già generato da RdMolDraw2D
      # impostazione del font Arial per Matplotlib
      font_prop = matplotlib.font_manager.FontProperties(fname='/content/arial.ttf', weight='bold')
      # Definizione finestra grafica Matplotlib
      fig, ax = plt.subplots(figsize=(10.8, 6))
      ax.imshow(img)
      for key, value in label_atom_coords.items():
        print(color_string(f'key: {key}',color='yellow'))
        if key == "and epimer at C*":
          # Imposta il testo con accenti e su più righe
          stringa_multilinea = "and epimer at C*\net l'épimère en C*\ny el epímero al C*"
          # Definisci la posizione e le differenze di coordinate
          x,y= value[0]
          # Inserisci il testo su più righe
          plt.text(x, y, stringa_multilinea, fontsize=12, fontproperties=font_prop, antialiased=True, ha='left', va='top')
        elif key == "and epimer at P":
          stringa_multilinea = "and epimer at P\net l'épimère en P\ny el epímero al P"
          x,y= value[0]
          plt.text(x, y, stringa_multilinea, fontsize=12, fontproperties=font_prop, antialiased=True, ha='left', va='top')
        elif key == "and enantiomer":
          stringa_multilinea = "and enantiomer\net énantiomère\ny enantiómero"
          x,y= value[0]
          plt.text(x, y, stringa_multilinea, fontsize=12, fontproperties=font_prop, antialiased=True, ha='left', va='top')
        elif key == "* and enantiomer":
          stringa_multilinea= "* and enantiomer\n* et énantiomère\n* y enantiómero"
          x,y= value[0]
          plt.text(x, y, stringa_multilinea, fontsize=12, fontproperties=font_prop, antialiased=True, ha='left', va='top')
        elif key == "or H<sup>+</sup>":
          stringa= r"$\mathrm{or\   H^+}$"
          x,y= value[0]
          plt.text(x, y, stringa, fontsize=12, fontproperties=font_prop, antialiased=True, ha='left', va='top')
        elif key == "-":
          stringa= "-"
          x,y= value[0]
          plt.text(x, y, stringa, fontsize=22, fontproperties=font_prop, antialiased=True, ha='left', va='top')
        elif key == "|":
          for graphic in root.iter('graphic'):
            # Verifica che il tipo di Graphic sia una parentesi quadra
            if graphic.get('BracketType') == 'Square':
                # Trova il nodo <objecttag> figlio
                objecttag = graphic.find('objecttag')
                if objecttag is not None:
                    # Trova il nodo <s> che contiene il numero
                    s_tag = objecttag.find('.//s')
                    if s_tag is not None:
                        numero = s_tag.text  # Numero trovato nel nodo <s>
                        print(f"Trovato numero: {numero}")
                        # Ora puoi salvare questo numero per usarlo più tardi
                        # Disegna parentesi quadra e il numero associato
          if len(value)==4: # Se gli atomi dummy delle parentesi sono 4
            for coord in value:
              x1,y1=coord
              print(color_string(f'coord_x: {x1}, coord_y: {y1}',color='yellow'))
              for coord_2 in value:
                x2,y2=coord_2
                print(color_string(f'coord_2_x: {x2}, coord_2_y: {y2}',color='cyan'))
                if x1-x2>-0.3 and x1-x2<0.3 and abs(y1-y2)>0.3:
                  print("disegno linea verticale")
                  if y1-y2>0:
                    print(color_string("L'atomo 1 è il punto superiore e L'atomo 2 è il punto inferiore",color='red'))
                    plt.plot([x1,x2],[y1,y2],color='black')
                    for coord_3 in value:
                      x3,y3=coord_3
                      if x3!=x1 and x3!=x2:
                        print(color_string(f'coord_3_x: {x3}, coord_3_y: {y3}',color='cyan'))
                        dif_x_1_3=x1-x3
                        dif_x_2_3=x2-x3
                        dif_y_1_3=y1-y3
                        dif_y_2_3=y2-y3
                        if dif_y_1_3 < 4.3 and dif_y_1_3 > -4.3:
                          print(color_string('atomo 1 e 3 sullo stesso piano y',color='green'))
                          if dif_x_1_3 < 0:
                            print(color_string('atomo 1 è a sinistra rispetto l\'atomo 3',color='green'))
                            plt.plot([x1,x1+abs(y1-y2)/10],[y1,y1],color='black')
                          else:
                            print(color_string('atomo 1 è a destra rispetto l\'atomo 3',color='green'))
                            plt.plot([x1,x1-abs(y1-y2)/10],[y1,y1],color='black')
                        else:
                          print(color_string('atomo 2 e 3 non sullo stesso piano y',color='green'))
                          if dif_x_2_3 < 0:
                            print(color_string('atomo 2 è a sinistra rispetto l\'atomo 3',color='green'))
                            plt.plot([x2,x2+abs(y1-y2)/10],[y2,y2],color='black')
                          else:
                            print(color_string('atomo 2 è a destra rispetto l\'atomo 3',color='green'))
                            plt.plot([x2,x2-abs(y1-y2)/10],[y2,y2],color='black')
            if numero:
              print(f'numero da aggiungere alle parentesi quadre: {numero}')
              x_max,y_max=0,0
              for x, y in value:
                if x > x_max:
                  x_max = x
                if y > y_max:
                  y_max = y
              x_min,y_min=1000,1000
              for x, y in value:
                if x < x_min:
                  x_min = x
                if y < y_min:
                  y_min = y
              plt.text(x_max+((x_max-x_min)/10), y_max+((y_max-y_min)/10), numero, fontsize=8, fontproperties=font_prop, ha='center', va='center')
      plt.axis('off')
      plt.savefig(nome_file, bbox_inches='tight' )
      # Chiudi la figura per prevenire la visualizzazione
      plt.close()
    print(f"Immagine generata")

In [26]:
# file CDX.zip

In [28]:
list_cdx=[]
for file in os.listdir('/content/molecole_cdx'):
  if file.endswith('.cdx'):
    list_cdx.append(file)
len(list_cdx)

158

In [29]:
# Crea le liste per ciascun tipo di errore
error_cdx_external_connection = []
error_boolean_length = []
error_tag_1101 = []
error_tag_1554 = []

for file in list_cdx:
    try:
        convert_cdx_to_cdxml(f'/content/molecole_cdx/{file}', f'/content/molecole_cdx/{file.split(".")[0]}.cdxml')
    except Exception as e:
        error_message = str(e)  # Converti l'eccezione in stringa per il confronto

        # Controlla quale errore è avvenuto e aggiungi il file alla lista corrispondente
        if "CDXExternalConnectionType should consist of 2 byte" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='red'))
            error_cdx_external_connection.append(file)
        elif "A Boolean should be exactly of length 1" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='yellow'))
            error_boolean_length.append(file)
        elif "Unknown property with tag_id 1101 found" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='green'))
            error_tag_1101.append(file)
        elif "Unknown property with tag_id 1554 found" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='blue'))
            error_tag_1554.append(file)
        else:
            print(f"Errore sconosciuto per il file {file}: {e}")

Errore per il file 13373_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13374_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13375_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13425_SH_GA.cdx: CDXExternalConnectionType should consist of 2 byte.


ERROR:pycdxml.chemdraw_types:No font with font id 25963 exists in the fonttable.


Errore per il file 13475.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13463_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13301_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13439_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13369_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13316_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13496.cdx: CDXExternalConnectionType should consist of 2 byte.


ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types

Errore per il file 13470_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13510.cdx: CDXExternalConnectionType should consist of 2 byte.
Errore per il file 13431.cdx: CDXExternalConnectionType should consist of 2 byte.
Errore per il file 13427.cdx: Unknown property with tag_id 1554 found.
Errore per il file 13390_1_AY.cdx: CDXExternalConnectionType should consist of 2 byte.
Errore per il file 13386_1_AY.cdx: Unknown property with tag_id 1101 found.


...

Errore per il file 13446_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13428_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13416_1_AY.cdx: Unknown property with tag_id 1101 found.


ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.


Errore per il file 13394_1_AY.cdx: Unknown property with tag_id 1101 found.


ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.


Errore per il file 13432_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13364_1_AY.cdx: Unknown property with tag_id 1101 found.
Errore per il file 13444_1_AY.cdx: Unknown property with tag_id 1101 found.


In [30]:
list_cdxml=[]
for file in os.listdir('/content/molecole_cdx'):
  if file.endswith('.cdxml'):
    list_cdxml.append(file)

In [31]:
os.makedirs('Cdxml_files_p132',exist_ok=True)

In [32]:
for file in list_cdxml:
  print(color_string(f'sposto il file: {file}',color='yellow'))
  move_fragment_into_group(f'/content/molecole_cdx/{file}',f'/content/Cdxml_files_p132/{file}')

sposto il file: 13473_1_AY.cdxml
Modifications saved to /content/Cdxml_files_p132/13473_1_AY.cdxml
sposto il file: 13458_1_AY.cdxml
Modifications saved to /content/Cdxml_files_p132/13458_1_AY.cdxml
sposto il file: 13522.cdxml
Modifications saved to /content/Cdxml_files_p132/13522.cdxml
sposto il file: 13443_1_AY.cdxml
Modifications saved to /content/Cdxml_files_p132/13443_1_AY.cdxml
sposto il file: 13357_1_AY.cdxml
Modifications saved to /content/Cdxml_files_p132/13357_1_AY.cdxml
sposto il file: 13307_1_AY.cdxml
Modifications saved to /content/Cdxml_files_p132/13307_1_AY.cdxml
sposto il file: 13532.cdxml
Moving fragment 100686 into group 99439
Fragment 100783 is already inside a group. No need to move.
Fragment 100784 is already inside a group. No need to move.
Fragment 100785 is already inside a group. No need to move.
Fragment 100686 is already inside a group. No need to move.
Fragment 100783 is already inside a group. No need to move.
Fragment 100784 is already inside a group. No ne

In [33]:
molobj={}
for name in list_cdxml:
  try:
    molobj[name]=Chem.rdmolfiles.MolsFromCDXMLFile(f'/content/Cdxml_files_p132/{name}',sanitize=True,removeHs=False)
  except Exception as e:
    error_message = str(e)
    print(color_string(f'molecola: {name} dà questo errore: {e}', color='red'))
molobj

[16:06:21] Failed to parse XML fragment 4199328 node: n attribute: NodeType: GenericNickname
[16:06:21] Failed to parse XML fragment 4199328 node: n attribute: NodeType: GenericNickname
[16:06:21] 

****
Invariant Violation
molzip: bond info not properly setup for bgn atom with label:101489
Violation occurred on line 995 in file /project/build/temp.linux-x86_64-cpython-311/rdkit/Code/GraphMol/ChemTransforms/MolFragmenter.cpp
Failed Expression: bond.a
----------
Stacktrace:
----------
****

[16:06:21] Failed fusion of fragment skipping... 101486
[16:06:21] Incomplete atom labelling, cannot make bond


{'13473_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397eff5a0>,),
 '13458_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397eff450>,),
 '13522.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397eff610>,
  <rdkit.Chem.rdchem.Mol at 0x78d397eff6f0>),
 '13443_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397eff760>,),
 '13357_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397eff7d0>,),
 '13307_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b0d0>,),
 '13532.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b140>,),
 '13376_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b1b0>,),
 '13667.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b220>,
  <rdkit.Chem.rdchem.Mol at 0x78d397f0b290>),
 '13366_1_AY.cdxml': (),
 '13354_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b300>,),
 '13482.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b370>,),
 '11257_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b3e0>,),
 '13455_1_AY.cdxml': (<rdkit.Chem.rdchem.Mol at 0x78d397f0b450>,),
 '13341_1_AY.cdxml': (<rdkit.Chem.rdche

In [34]:
keys_to_remove = []
for key, value in molobj.items():
    print(color_string(f'applico l\'unione della molecola {key}', color='yellow'))
    # Verifica se ci sono più frammenti molecolari
    if len(value) > 1:
        print(f'La molecola {key} ha {len(value)} frammenti molecolari da unire')
        # Unisci i frammenti uno alla volta
        combined_mol = value[0]  # Prendi il primo oggetto molecolare dalla lista
        # Itera su tutti gli altri frammenti e combinali
        for frag in value[1:]:
            combined_mol = Chem.CombineMols(combined_mol, frag)
        # Aggiorna il dizionario con la molecola combinata (non più una lista)
        molobj[key] = combined_mol
        # Verifica la nuova molecola combinata
        print(Chem.MolToSmiles(molobj[key]))
    else:
        if value:
        # Se c'è solo un frammento, aggiorna direttamente con quell'oggetto molecolare
          molobj[key] = value[0]
          print(f'La molecola {key} ha un solo frammento, nessuna unione necessaria.')
        else:
          keys_to_remove.append(key)
# Rimuove le chiavi che hanno liste vuote
for key in keys_to_remove:
    print(color_string(f'Rimuovo la molecola {key} che ha una lista vuota', color='red'))
    del molobj[key]

applico l'unione della molecola 13473_1_AY.cdxml
La molecola 13473_1_AY.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13458_1_AY.cdxml
La molecola 13458_1_AY.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13522.cdxml
La molecola 13522.cdxml ha 3 frammenti molecolari da unire
CCCC.CCCC.CCCC
applico l'unione della molecola 13443_1_AY.cdxml
La molecola 13443_1_AY.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13357_1_AY.cdxml
La molecola 13357_1_AY.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13307_1_AY.cdxml
La molecola 13307_1_AY.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13532.cdxml
La molecola 13532.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13376_1_AY.cdxml
La molecola 13376_1_AY.cdxml ha un solo frammento, nessuna unione necessaria.
app

In [35]:
for key in molobj.keys():
    print(color_string(f'applico abbreviazioni sulla molecola: {key}', color='yellow'))
    try:
      mol_modificata = abbrevs_molecule(molobj[key])
      if mol_modificata:  # Verifica se mol_modificata non è None
        molobj[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
      # Passa la molecola corrente
      else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))
    except Exception as e:
      print(color_string(f'molecola: {key} dà questo errore: {e}', color='red'))

applico abbreviazioni sulla molecola: 13473_1_AY.cdxml
applico abbreviazioni sulla molecola: 13458_1_AY.cdxml
applico abbreviazioni sulla molecola: 13522.cdxml
applico abbreviazioni sulla molecola: 13443_1_AY.cdxml
applico abbreviazioni sulla molecola: 13357_1_AY.cdxml
applico abbreviazioni sulla molecola: 13307_1_AY.cdxml
applico abbreviazioni sulla molecola: 13532.cdxml
applico abbreviazioni sulla molecola: 13376_1_AY.cdxml
applico abbreviazioni sulla molecola: 13667.cdxml
applico abbreviazioni sulla molecola: 13354_1_AY.cdxml
applico abbreviazioni sulla molecola: 13482.cdxml
applico abbreviazioni sulla molecola: 11257_9.cdxml
applico abbreviazioni sulla molecola: 13455_1_AY.cdxml
applico abbreviazioni sulla molecola: 13341_1_AY.cdxml
applico abbreviazioni sulla molecola: 13472_1_AY.cdxml
applico abbreviazioni sulla molecola: 13535.cdxml
applico abbreviazioni sulla molecola: 13514.cdxml
applico abbreviazioni sulla molecola: 13541.cdxml
applico abbreviazioni sulla molecola: 11261_9.cd

[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 atoms ignored
[16:06:30] abbreviation with <2 at

In [36]:
for key in molobj.keys():
    print(color_string(f'applico delete_atom_with_warnings sulla molecola: {key}', color='yellow'))
    # Passa la molecola corrente e il percorso del file cdxml alla funzione
    mol_modificata = delete_atom_with_warning_info(molobj[key], pathcdxml=f'/content/molecole_cdx/{key}')
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))

applico delete_atom_with_warnings sulla molecola: 13473_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13458_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13522.cdxml
applico delete_atom_with_warnings sulla molecola: 13443_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13357_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13307_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13532.cdxml
applico delete_atom_with_warnings sulla molecola: 13376_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13667.cdxml
applico delete_atom_with_warnings sulla molecola: 13354_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13482.cdxml
applico delete_atom_with_warnings sulla molecola: 11257_9.cdxml
applico delete_atom_with_warnings sulla molecola: 13455_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13341_1_AY.cdxml
applico delete_atom_with_warnings sulla molecola: 13472_1_AY.cdxml
applico delete_ato

In [37]:
for mol in molobj.keys():
  print(color_string(f'applico Retain_stereo sulla molecola: {mol}',color='yellow'))
  Retain_Stereo_and_map_fragments(molobj[mol],f'/content/molecole_cdx/{mol}',verboso=False)

applico Retain_stereo sulla molecola: 13473_1_AY.cdxml
applico Retain_stereo sulla molecola: 13458_1_AY.cdxml
legame tra 154 e 322, Wedge
le coordinate dell'atomo finale sono: (182.21, 22.2)
le coordinate dell'atomo iniziale sono: (179.24, 36.17)
atomo iniziale: 1
atomo finale: 24
ci sono legami stereo Wedge tra 1 e 24 imposto STEREOANY e BEGINWEDGE
legame tra 154 e 325, Dash
le coordinate dell'atomo finale sono: (168.62, 26.61)
le coordinate dell'atomo iniziale sono: (179.24, 36.17)
atomo iniziale: 1
atomo finale: 25
ci sono legami stereo Dash tra 1 e 25 imposto STEREOANY e BEGINDASH
applico Retain_stereo sulla molecola: 13522.cdxml
applico Retain_stereo sulla molecola: 13443_1_AY.cdxml
legame tra 129 e 131, Wedge
le coordinate dell'atomo finale sono: (275.23, 112.11)
le coordinate dell'atomo iniziale sono: (262.86, 104.96)
atomo iniziale: 13
atomo finale: 14
ci sono legami stereo Wedge tra 13 e 14 imposto STEREOANY e BEGINWEDGE
legame tra 131 e 259, Dash
le coordinate dell'atomo fina

In [38]:
for key in molobj.keys():
    print(color_string(f'applico reset_radical_for_dative_bonds sulla molecola: {key}', color='red'))
    reset_radical_for_dative_bonds(molobj[key])

applico reset_radical_for_dative_bonds sulla molecola: 13473_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13458_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13522.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13443_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13357_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13307_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13532.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13376_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13667.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13354_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13482.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 11257_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13455_1_AY.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13341_1_AY.cdxml
applico reset_r

In [39]:
for key in molobj.keys():
    print(color_string(f'applico add_label sulla molecola: {key}', color='yellow'))
    try:# Passa la molecola corrente e il percorso del file cdxml alla funzione
      mol_modificata = add_label(molobj[key], pathcdxml=f'/content/molecole_cdx/{key}',verboso=False)
      if mol_modificata:  # Verifica se mol_modificata non è None
          molobj[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
      else:
          print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))
    except Exception as e:
      print(color_string(f'molecola: {key} dà questo errore: {e}', color='magenta'))

applico add_label sulla molecola: 13473_1_AY.cdxml
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
applico add_label sulla molecola: 13458_1_AY.cdxml
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non trovato.
stringa da aggiungere = OH
Tag <s> non t

In [40]:
os.makedirs('/content/molecole_OMS',exist_ok=True)

In [42]:
for mol in molobj.keys():
  try:
    print(color_string(f'disegnando la molecola: {mol}',color='magenta'))
    draw_OMS_molecule(molobj[mol],pathcdxml=f'/content/molecole_cdx/{mol}',nome_file=f'/content/molecole_OMS/{mol}.png')
  except Exception as e:
    print(color_string(f'molecola: {mol} dà questo errore: {e}', color='magenta'))

disegnando la molecola: 13473_1_AY.cdxml
Immagine generata
disegnando la molecola: 13458_1_AY.cdxml
Atomo 20 ha l'etichetta: CF<sub>3</sub>
Immagine generata
disegnando la molecola: 13522.cdxml
Immagine generata
disegnando la molecola: 13443_1_AY.cdxml
Immagine generata
disegnando la molecola: 13357_1_AY.cdxml
Immagine generata
disegnando la molecola: 13307_1_AY.cdxml
Immagine generata
disegnando la molecola: 13532.cdxml
Atomo 4 ha l'etichetta: CO<sub>2</sub>H
Atomo 7 ha l'etichetta: CO<sub>2</sub>H
Atomo 10 ha l'etichetta: CO<sub>2</sub>H
Atomo 11 ha l'etichetta: |
Atomo 12 ha l'etichetta: |
Atomo 13 ha l'etichetta: |
Atomo 14 ha l'etichetta: |
label_atom_coords {'|': [(465.74866403608945, 347.9831212739707), (465.74866403608945, 478.4233099292787), (256.9455438628576, 478.4233099292787), (256.9455438628576, 347.9831212739707)]}
atomi_da_cancellare [11, 12, 13, 14]
Atomo linea o meno 11 rimosso.
Atomo linea o meno 12 rimosso.
Atomo linea o meno 13 rimosso.
Atomo linea o meno 14 rimoss

In [43]:
!zip -r /content/molecole_OMS.zip /content/molecole_OMS

  adding: content/molecole_OMS/ (stored 0%)
  adding: content/molecole_OMS/13514.cdxml.png (deflated 15%)
  adding: content/molecole_OMS/13322_1_AY.cdxml.png (deflated 14%)
  adding: content/molecole_OMS/13423_1_AY.cdxml.png (deflated 17%)
  adding: content/molecole_OMS/13302_1_AY.cdxml.png (deflated 21%)
  adding: content/molecole_OMS/13460_1_AY.cdxml.png (deflated 19%)
  adding: content/molecole_OMS/13447_1_AY.cdxml.png (deflated 14%)
  adding: content/molecole_OMS/13459_1_AY.cdxml.png (deflated 18%)
  adding: content/molecole_OMS/13462_1_AY.cdxml.png (deflated 14%)
  adding: content/molecole_OMS/13313_1_AY.cdxml.png (deflated 15%)
  adding: content/molecole_OMS/13435_1_AY.cdxml.png (deflated 16%)
  adding: content/molecole_OMS/11204_9.cdxml.png (deflated 14%)
  adding: content/molecole_OMS/13443_1_AY.cdxml.png (deflated 15%)
  adding: content/molecole_OMS/13519.cdxml.png (deflated 14%)
  adding: content/molecole_OMS/13299_1_AY.cdxml.png (deflated 15%)
  adding: content/molecole_OMS/

In [ ]:
# file p130.zip

In [ ]:
list_cdx_p130=[]
for file in os.listdir('/content/p130'):
  if file.endswith('.cdx'):
    list_cdx_p130.append(file)
len(list_cdx_p130)

91

In [ ]:
# Crea le liste per ciascun tipo di errore
error_cdx_external_connection_p130 = []
error_boolean_length_p130 = []
error_tag_1101_p130 = []
error_tag_1554_p130 = []
for file in list_cdx_p130:
    try:
        convert_cdx_to_cdxml(f'/content/p130/{file}', f'/content/p130/{file.split(".")[0]}.cdxml')
    except Exception as e:
        error_message = str(e)  # Converti l'eccezione in stringa per il confronto

        # Controlla quale errore è avvenuto e aggiungi il file alla lista corrispondente
        if "CDXExternalConnectionType should consist of 2 byte" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='red'))
            error_cdx_external_connection_p130.append(file)
        elif "A Boolean should be exactly of length 1" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='yellow'))
            error_boolean_length_p130.append(file)
        elif "Unknown property with tag_id 1101 found" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='green'))
            error_tag_1101_p130.append(file)
        elif "Unknown property with tag_id 1554 found" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='blue'))
            error_tag_1554_p130.append(file)
        else:
            print(f"Errore sconosciuto per il file {file}: {e}")

ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types

Errore per il file 12866_9.cdx: CDXExternalConnectionType should consist of 2 byte.


ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types

Errore per il file 12956_9.cdx: CDXExternalConnectionType should consist of 2 byte.


ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types

Errore per il file 12937_9.cdx: CDXExternalConnectionType should consist of 2 byte.


ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.
ERROR:pycdxml.chemdraw_types:No font with font id 28517 exists in the fonttable.


In [ ]:
list_cdxml_p130=[]
for file in os.listdir('/content/p130'):
  if file.endswith('.cdxml'):
    list_cdxml_p130.append(file)
len(list_cdxml_p130)

88

In [ ]:
os.makedirs('Cdxml_files_p130',exist_ok=True)

In [ ]:
for file in list_cdxml_p130:
  print(color_string(f'sposto il file: {file}',color='yellow'))
  move_fragment_into_group(f'/content/p130/{file}',f'/content/Cdxml_files_p130/{file}')

sposto il file: 12938_9.cdxml
Fragment 95406 is already inside a group. No need to move.
Fragment 95534 is already inside a group. No need to move.
Modifications saved to /content/Cdxml_files_p130/12938_9.cdxml
sposto il file: 12841_9.cdxml
Modifications saved to /content/Cdxml_files_p130/12841_9.cdxml
sposto il file: 13027_9.cdxml
Modifications saved to /content/Cdxml_files_p130/13027_9.cdxml
sposto il file: 12907_9.cdxml
Fragment 93710 is already inside a group. No need to move.
Fragment 93795 is already inside a group. No need to move.
Fragment 93796 is already inside a group. No need to move.
Modifications saved to /content/Cdxml_files_p130/12907_9.cdxml
sposto il file: 12844_9.cdxml
Fragment 1381 is already inside a group. No need to move.
Fragment 1615 is already inside a group. No need to move.
Modifications saved to /content/Cdxml_files_p130/12844_9.cdxml
sposto il file: 12871_9.cdxml
Fragment 42822 is already inside a group. No need to move.
Fragment 43118 is already inside a 

In [ ]:
molobj_p130={}
for name in list_cdxml_p130:
  try:
    molobj_p130[name]=Chem.rdmolfiles.MolsFromCDXMLFile(f'/content/Cdxml_files_p130/{name}',sanitize=True,removeHs=False)
  except Exception as e:
    error_message = str(e)
    print(color_string(f'molecola: {name} dà questo errore: {e}', color='red'))
molobj_p130

[03:58:04] Explicit valence for atom # 31 Sn, 6, is greater than permitted
[03:58:04] CDXMLParser: failed sanitizing skipping fragment 103853
[03:58:04] Explicit valence for atom # 31 Sn, 6, is greater than permitted
[03:58:04] CDXMLParser: failed sanitizing skipping fragment 103853
[03:58:04] 

****
Invariant Violation
molzip: bond info not properly setup for bgn atom with label:93260
Violation occurred on line 995 in file /project/build/temp.linux-x86_64-cpython-311/rdkit/Code/GraphMol/ChemTransforms/MolFragmenter.cpp
Failed Expression: bond.a
----------
Stacktrace:
----------
****

[03:58:04] Failed fusion of fragment skipping... 93259


{'12938_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbee30>,),
 '12841_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbd7e0>,),
 '13027_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbd3f0>,),
 '12907_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbf920>,),
 '12844_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbec00>,),
 '12871_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbdbd0>,),
 '12910_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbfed0>,),
 '12861_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbd690>,),
 '12989_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbcc80>,),
 '12867_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbc580>,),
 '12939_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbc5f0>,),
 '12974_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbd150>,),
 '12908_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbc660>,),
 '12897_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbc6d0>,),
 '12934_9.cdxml': (<rdkit.Chem.rdchem.Mol at 0x7b1927bbeb90>,),
 '13293_9.cdxml': (<rdkit.Chem.rdchem.Mo

In [ ]:
keys_to_remove = []
for key, value in molobj_p130.items():
    print(color_string(f'applico l\'unione della molecola {key}', color='yellow'))
    # Verifica se ci sono più frammenti molecolari
    if len(value) > 1:
        print(f'La molecola {key} ha {len(value)} frammenti molecolari da unire')
        # Unisci i frammenti uno alla volta
        combined_mol = value[0]  # Prendi il primo oggetto molecolare dalla lista
        # Itera su tutti gli altri frammenti e combinali
        for frag in value[1:]:
            combined_mol = Chem.CombineMols(combined_mol, frag)
        # Aggiorna il dizionario con la molecola combinata (non più una lista)
        molobj_p130[key] = combined_mol
        # Verifica la nuova molecola combinata
        print(Chem.MolToSmiles(molobj_p130[key]))
    else:
        if value:
        # Se c'è solo un frammento, aggiorna direttamente con quell'oggetto molecolare
          molobj_p130[key] = value[0]
          print(f'La molecola {key} ha un solo frammento, nessuna unione necessaria.')
        else:
          keys_to_remove.append(key)
# Rimuovi le chiavi che hanno liste vuote
for key in keys_to_remove:
    print(color_string(f'Rimuovo la molecola {key} che ha una lista vuota', color='red'))
    del molobj_p130[key]

applico l'unione della molecola 12938_9.cdxml
La molecola 12938_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 12841_9.cdxml
La molecola 12841_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 13027_9.cdxml
La molecola 13027_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 12907_9.cdxml
La molecola 12907_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 12844_9.cdxml
La molecola 12844_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 12871_9.cdxml
La molecola 12871_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 12910_9.cdxml
La molecola 12910_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 12861_9.cdxml
La molecola 12861_9.cdxml ha un solo frammento, nessuna unione necessaria.
applico l'unione della molecola 

In [ ]:
for key in molobj_p130.keys():
    # Passa la molecola corrente
    mol_modificata = abbrevs_molecule(molobj_p130[key])
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj_p130[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))


[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 atoms ignored
[03:58:06] abbreviation with <2 at

In [ ]:
for key in molobj_p130.keys():
    print(color_string(f'applico delete_atom_with_warnings sulla molecola: {key}', color='yellow'))
    # Passa la molecola corrente e il percorso del file cdxml alla funzione
    mol_modificata = delete_atom_with_warning_info(molobj_p130[key], pathcdxml=f'/content/p130/{key}')
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj_p130[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))

applico delete_atom_with_warnings sulla molecola: 12938_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12841_9.cdxml
applico delete_atom_with_warnings sulla molecola: 13027_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12907_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12844_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12871_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12910_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12861_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12989_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12867_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12939_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12974_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12908_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12897_9.cdxml
applico delete_atom_with_warnings sulla molecola: 12934_9.cdxml
applico delete_atom_with_warnings sulla 

In [ ]:
for mol in molobj_p130.keys():
  print(color_string(f'applico Retain_stereo sulla molecola: {mol}',color='yellow'))
  Retain_Stereo_and_map_fragments(molobj_p130[mol],f'/content/p130/{mol}',verboso=False)

applico Retain_stereo sulla molecola: 12938_9.cdxml
legame tra 95417 e 95436, Wedge
le coordinate dell'atomo finale sono: (202.59, 144.65)
le coordinate dell'atomo iniziale sono: (216.79, 144.65)
atomo iniziale: 10
atomo finale: 19
ci sono legami stereo Wedge tra 10 e 19 imposto STEREOANY e BEGINWEDGE
legame tra 95449 e 95415, Wedge
le coordinate dell'atomo finale sono: (229.09, 109.15)
le coordinate dell'atomo iniziale sono: (241.38, 102.05)
atomo iniziale: 32
atomo finale: 8
ci sono legami stereo Wedge tra 32 e 8 imposto STEREOANY e BEGINWEDGE
legame tra 95417 e 95453, Dash
le coordinate dell'atomo finale sono: (209.69, 156.94)
le coordinate dell'atomo iniziale sono: (216.79, 144.65)
atomo iniziale: 10
atomo finale: 36
ci sono legami stereo Dash tra 10 e 36 imposto STEREOANY e BEGINDASH
legame tra 95449 e 95454, Dash
le coordinate dell'atomo finale sono: (229.13, 94.97)
le coordinate dell'atomo iniziale sono: (241.38, 102.05)
atomo iniziale: 32
atomo finale: 37
ci sono legami stereo 

In [ ]:
for key in molobj_p130.keys():
    print(color_string(f'applico reset_radical_for_dative_bonds sulla molecola: {key}', color='red'))
    reset_radical_for_dative_bonds(molobj_p130[key])


applico reset_radical_for_dative_bonds sulla molecola: 12938_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12841_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 13027_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12907_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12844_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12871_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12910_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12861_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12989_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12867_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12939_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12974_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12908_9.cdxml
applico reset_radical_for_dative_bonds sulla molecola: 12897_9.cdxml
applico reset_radical_for_dative_b

In [ ]:
# Verifica che add_ions restituisca una molecola aggiornata
for key in molobj_p130.keys():
    print(color_string(f'applico add_ions sulla molecola: {key}', color='yellow'))
    # Passa la molecola corrente e il percorso del file cdxml alla funzione
    mol_modificata = add_ions(molobj_p130[key], pathcdxml=f'/content/p130/{key}')
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj_p130[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))

applico add_ions sulla molecola: 12938_9.cdxml
applico add_ions sulla molecola: 12841_9.cdxml
applico add_ions sulla molecola: 13027_9.cdxml
applico add_ions sulla molecola: 12907_9.cdxml
applico add_ions sulla molecola: 12844_9.cdxml
applico add_ions sulla molecola: 12871_9.cdxml
applico add_ions sulla molecola: 12910_9.cdxml
applico add_ions sulla molecola: 12861_9.cdxml
applico add_ions sulla molecola: 12989_9.cdxml
applico add_ions sulla molecola: 12867_9.cdxml
applico add_ions sulla molecola: 12939_9.cdxml
applico add_ions sulla molecola: 12974_9.cdxml
applico add_ions sulla molecola: 12908_9.cdxml
applico add_ions sulla molecola: 12897_9.cdxml
applico add_ions sulla molecola: 12934_9.cdxml
applico add_ions sulla molecola: 13293_9.cdxml
applico add_ions sulla molecola: 12892_9.cdxml
applico add_ions sulla molecola: 12884_9.cdxml
applico add_ions sulla molecola: 12618_9.cdxml
applico add_ions sulla molecola: 12839_9.cdxml
applico add_ions sulla molecola: 12398_9.cdxml
applico add_i

In [ ]:
for key in molobj_p130.keys():
    print(color_string(f'applico add_label sulla molecola: {key}', color='yellow'))
    try:# Passa la molecola corrente e il percorso del file cdxml alla funzione
      mol_modificata = add_label(molobj_p130[key], pathcdxml=f'/content/p130/{key}',verboso=False)
      if mol_modificata:  # Verifica se mol_modificata non è None
          molobj_p130[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
      else:
          print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))
    except Exception as e:
      print(color_string(f'molecola: {key} dà questo errore: {e}', color='magenta'))

applico add_label sulla molecola: 12938_9.cdxml
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
applico add_label sulla molecola: 12841_9.cdxml
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da aggiungere = N
Tag <s> non trovato.
stringa da 

In [ ]:
os.makedirs('/content/disegni_p130',exist_ok=True)

In [ ]:
for mol in molobj_p130.keys():
  try:
    print(color_string(f'disegnando la molecola: {mol}',color='magenta'))
    disegnala_come_voglio(molobj_p130[mol],nome_file=f'/content/disegni_p130/{mol}.png',pathcdxml=f'/content/p130/{mol}')
  except Exception as e:
    print(color_string(f'molecola: {mol} dà questo errore: {e}', color='magenta'))

disegnando la molecola: 12938_9.cdxml
Atomo 18 ha l'etichetta: CF<sub>3</sub>
Immagine generata
disegnando la molecola: 12841_9.cdxml
Atomo 28 ha l'etichetta: OCH<sub>3</sub>
Immagine generata
disegnando la molecola: 13027_9.cdxml
Atomo 30 ha l'etichetta: CN
Immagine generata
disegnando la molecola: 12907_9.cdxml
Atomo 0 ha l'etichetta: CO<sub>2</sub>H
Atomo 19 ha l'etichetta: OCH<sub>3</sub>
Immagine generata
disegnando la molecola: 12844_9.cdxml
Atomo 6 ha l'etichetta: CN
Immagine generata
disegnando la molecola: 12871_9.cdxml
Atomo 22 ha l'etichetta: CF<sub>3</sub>
Immagine generata
disegnando la molecola: 12910_9.cdxml
Atomo 15 ha l'etichetta: NO<sub>2</sub>
Immagine generata
disegnando la molecola: 12861_9.cdxml
Atomo 3 ha l'etichetta: CO<sub>2</sub>H
Atomo 19 ha l'etichetta: CN
Immagine generata
disegnando la molecola: 12989_9.cdxml
Immagine generata
disegnando la molecola: 12867_9.cdxml
Atomo 6 ha l'etichetta: CN
Immagine generata
disegnando la molecola: 12939_9.cdxml
Atomo 10 h

In [ ]:
!zip -r /content/disegni_p130.zip /content/disegni_p130

In [ ]:
# PROPOSED LIST 131

In [ ]:
list_cdx_p131=[]
for file in os.listdir('/content/p131'):
  if file.endswith('.cdx'):
    list_cdx_p131.append(file)
len(list_cdx_p131)

In [ ]:
# Crea le liste per ciascun tipo di errore
error_cdx_external_connection_p131 = []
error_boolean_length_p131 = []
error_tag_1101_p131 = []
error_tag_1554_p131 = []
for file in list_cdx_p131:
    print(color_string(f'elaborando il file: {file}',color='yellow'))
    try:
        convert_cdx_to_cdxml(f'/content/p131/{file}', f'/content/p131/{file.split(".")[0]}.cdxml')
    except Exception as e:
        error_message = str(e)  # Converti l'eccezione in stringa per il confronto

        # Controlla quale errore è avvenuto e aggiungi il file alla lista corrispondente
        if "CDXExternalConnectionType should consist of 2 byte" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='red'))
            error_cdx_external_connection_p131.append(file)
        elif "A Boolean should be exactly of length 1" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='yellow'))
            error_boolean_length_p131.append(file)
        elif "Unknown property with tag_id 1101 found" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='green'))
            error_tag_1101_p131.append(file)
        elif "Unknown property with tag_id 1554 found" in error_message:
            print(color_string(f"Errore per il file {file}: {e}",color='blue'))
            error_tag_1554_p131.append(file)
        else:
            print(f"Errore sconosciuto per il file {file}: {e}")

In [ ]:
list_cdxml_p131=[]
for file in os.listdir('/content/p131'):
  if file.endswith('.cdxml'):
    list_cdxml_p131.append(file)
len(list_cdxml_p131)

In [ ]:
os.makedirs('Cdxml_files_p131',exist_ok=True)

In [ ]:
for file in list_cdxml_p131:
  print(color_string(f'sposto il file: {file}',color='yellow'))
  move_fragment_into_group(f'/content/p131/{file}',f'/content/Cdxml_files_p131/{file}')


In [ ]:
molobj_p131={}
for name in list_cdxml_p131:
  print(color_string(f'elaborando la molecola: {name}',color='yellow'))
  try:
    molobj_p131[name]=Chem.rdmolfiles.MolsFromCDXMLFile(f'/content/Cdxml_files_p131/{name}',sanitize=True,removeHs=False)
  except Exception as e:
    error_message = str(e)
    print(color_string(f'molecola: {name} dà questo errore: {e}', color='red'))
len(molobj_p131)

In [ ]:
keys_to_remove = []
for key, value in molobj_p131.items():
    print(color_string(f'applico l\'unione della molecola {key}', color='yellow'))
    # Verifica se ci sono più frammenti molecolari
    if len(value) > 1:
        print(f'La molecola {key} ha {len(value)} frammenti molecolari da unire')
        # Unisci i frammenti uno alla volta
        combined_mol = value[0]  # Prendi il primo oggetto molecolare dalla lista
        # Itera su tutti gli altri frammenti e combinali
        for frag in value[1:]:
            combined_mol = Chem.CombineMols(combined_mol, frag)
        # Aggiorna il dizionario con la molecola combinata (non più una lista)
        molobj_p131[key] = combined_mol
        # Verifica la nuova molecola combinata
        print(Chem.MolToSmiles(molobj_p131[key]))
    else:
        if value:
        # Se c'è solo un frammento, aggiorna direttamente con quell'oggetto molecolare
          molobj_p131[key] = value[0]
          print(f'La molecola {key} ha un solo frammento, nessuna unione necessaria.')
        else:
          keys_to_remove.append(key)
# Rimuovi le chiavi che hanno liste vuote
for key in keys_to_remove:
    print(color_string(f'Rimuovo la molecola {key} che ha una lista vuota', color='red'))
    del molobj_p131[key]

In [ ]:
for key in molobj_p131.keys():
    # Passa la molecola corrente
    mol_modificata = abbrevs_molecule(molobj_p131[key])
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj_p131[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))


In [ ]:
for key in molobj_p131.keys():
    print(color_string(f'applico delete_atom_with_warnings sulla molecola: {key}', color='yellow'))
    # Passa la molecola corrente e il percorso del file cdxml alla funzione
    mol_modificata = delete_atom_with_warning_info(molobj_p131[key], pathcdxml=f'/content/p131/{key}')
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj_p131[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))

In [ ]:
for mol in molobj_p131.keys():
  print(color_string(f'applico Retain_stereo sulla molecola: {mol}',color='yellow'))
  Retain_Stereo_and_map_fragments(molobj_p131[mol],f'/content/p131/{mol}',verboso=False)

In [ ]:
for key in molobj_p131.keys():
    print(color_string(f'applico reset_radical_for_dative_bonds sulla molecola: {key}', color='red'))
    reset_radical_for_dative_bonds(molobj_p131[key])


In [ ]:
for key in molobj_p131.keys():
    print(color_string(f'applico add_ions sulla molecola: {key}', color='yellow'))
    # Passa la molecola corrente e il percorso del file cdxml alla funzione
    mol_modificata = add_ions(molobj_p131[key], pathcdxml=f'/content/p131/{key}')
    if mol_modificata:  # Verifica se mol_modificata non è None
        molobj_p131[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
    else:
        print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))

In [ ]:
for key in molobj_p131.keys():
    print(color_string(f'applico add_label sulla molecola: {key}', color='yellow'))
    try:# Passa la molecola corrente e il percorso del file cdxml alla funzione
      mol_modificata = add_label(molobj_p131[key], pathcdxml=f'/content/p131/{key}',verboso=False)
      if mol_modificata:  # Verifica se mol_modificata non è None
          molobj_p131[key] = mol_modificata  # Aggiorna il dizionario con la molecola modificata
      else:
          print(color_string(f'Nessuna modifica apportata alla molecola: {key}', color='red'))
    except Exception as e:
      print(color_string(f'molecola: {key} dà questo errore: {e}', color='magenta'))

In [ ]:
os.makedirs('/content/disegni_p131',exist_ok=True)

In [ ]:
for mol in molobj_p131.keys():
  try:
    print(color_string(f'disegnando la molecola: {mol}', color='magenta'))
    disegnala_come_voglio(molobj_p131[mol],pathcdxml=f'/content/p131/{mol}',nome_file=f'/content/disegni_p131/{mol}.png', verboso=False)
  except Exception as e:
    print(color_string(f'molecola: {mol} dà questo errore: {e}', color='magenta'))

In [ ]:
!zip -r /content/disegni_p131.zip /content/disegni_p131